# 第2章 商品推薦を実現する数理

このNotebookでは、レコメンドシステムを実現するための数理的基礎を学びます。

**キーワード**：協調フィルタリング、行列因子分解、最小二乗法、勾配降下法、評価値行列、コサイン類似度、指示関数、損失関数、内積、三角関数、ベクトル、微分、偏微分、行列

## 目次
- 2-1 はじめに
- 2-2 商品の評価を数理的に表現する ー評価値行列ー
- 2-3 評価値の予測を数理モデルで実現する ー協調フィルタリングと行列因子分解ー
- 2-4 ユーザー同士の類似度で予測値を推計する ー内積の定理とコサイン類似度ー
- 2-5 コサイン類似度の意味を考える ー三角関数ー
- 2-6 コサイン類似度を複数のアイテムに適用する ー多次元への拡張ー
- 2-7 コサイン類似度を改良する ー中心化ー
- 2-8 コサイン類似度を計算する ー指示関数ー
- 2-9 欠損値を推計する数理モデルを設計し、計算を実行する
- 2-10 アイテム同士の類似度で予測値を推計する
- 2-11 ユーザー目線で数理モデルを再考する ーセレンディピティー
- 2-12 課題解決のために数理モデルを変更する ー行列因子分解ー
- 2-13 評価値の推計を最適化問題に置き換える ー残差行列と誤差ー

## 環境設定

In [ ]:
# 日本語フォントのインストール（Google Colab用）
!pip install japanize-matplotlib -q

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import japanize_matplotlib

# グラフの設定
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

---
# 2-1 はじめに

私たちの生活は**レコメンド**に溢れています。

- オンラインショッピングの「おすすめの商品」
- YouTubeの「あなたへのおすすめ動画」
- SNSの「興味のある投稿」

レコメンドは、レコメンドのために作られた**数理モデル**が膨大なデータを処理することで実現します。

本章では、レコメンドを実現する基礎的な数学的手法に焦点を当てて解説します。具体的には、**協調フィルタリング**と**行列因子分解**と呼ばれる数理的手法を中心に解説し、いかにして数理モデルが私たちの「好み」を理解し、購入する可能性が高い商品やコンテンツをレコメンドしているのかを解き明かします。

---
# 2-2 商品の評価を数理的に表現する ー評価値行列ー

レコメンドを数理モデルで実現するために、まず**評価値行列**を作成します。

## ベクトルによる表現

ベクトルは数値の順序付きリストです。

$$\vec{r} = \begin{pmatrix} r_1 \\ r_2 \\ \vdots \\ r_n \end{pmatrix}, \quad \vec{r}^T = (r_1, r_2, \cdots, r_n)$$

- 左側の式は縦の「列方向」に数値が並んでいるので**列ベクトル**
- 右側の式は横の「行方向」に数値が並んでいるので**行ベクトル**
- $^T$ は**転置**を表す記号で、行と列の方向を変える操作を指す

In [ ]:
# ユーザーごとの評価値をベクトルで表現
# 図2.3の例：4人のユーザーが4つのアイテムを評価
# 0は欠損値（未評価）を表す

# 行ベクトル表現
R_u1 = np.array([2, 3, 0, 5])  # user1の評価
R_u2 = np.array([2, 5, 0, 5])  # user2の評価
R_u3 = np.array([0, 3, 4, 4])  # user3の評価
R_u4 = np.array([4, 2, 3, 0])  # user4の評価

print("=== ユーザーごとの評価ベクトル（行ベクトル） ===")
print(f"R_u1 = {R_u1}")
print(f"R_u2 = {R_u2}")
print(f"R_u3 = {R_u3}")
print(f"R_u4 = {R_u4}")

In [ ]:
# 列ベクトル表現（アイテムごとの評価）
R_i1 = np.array([[2], [2], [0], [4]])  # item1の評価
R_i2 = np.array([[3], [5], [3], [2]])  # item2の評価
R_i3 = np.array([[0], [0], [4], [3]])  # item3の評価
R_i4 = np.array([[5], [5], [4], [0]])  # item4の評価

print("=== アイテムごとの評価ベクトル（列ベクトル） ===")
print(f"R_i1 = {R_i1.flatten()}")
print(f"R_i2 = {R_i2.flatten()}")
print(f"R_i3 = {R_i3.flatten()}")
print(f"R_i4 = {R_i4.flatten()}")

## 評価値行列 R

行ベクトル $R_{u_1} \sim R_{u_4}$ を列方向（縦）に並べると、評価値を1つにまとめた**行列**で表現できます。これを**評価値行列**と呼び、$R$と表記します。

$$R = \begin{pmatrix} R_{u_1} \\ R_{u_2} \\ R_{u_3} \\ R_{u_4} \end{pmatrix} = \begin{pmatrix} 2 & 3 & 0 & 5 \\ 2 & 5 & 0 & 5 \\ 0 & 3 & 4 & 4 \\ 4 & 2 & 3 & 0 \end{pmatrix}$$

- $R$ は Rating（評価）の頭文字
- 0 は未評価、つまり**欠損値**を示す
- user1のitem1に対する評価値を $r_{1,1}$ と表記する

一般化すると、ユーザーの総数を $U$、アイテムの総数を $I$ とすると：

$$R = \begin{pmatrix} r_{1,1} & r_{1,2} & \cdots & r_{1,I} \\ r_{2,1} & r_{2,2} & \cdots & r_{2,I} \\ \vdots & \vdots & \ddots & \vdots \\ r_{U,1} & r_{U,2} & \cdots & r_{U,I} \end{pmatrix}$$

In [ ]:
# 評価値行列の作成
R = np.array([
    [2, 3, 0, 5],
    [2, 5, 0, 5],
    [0, 3, 4, 4],
    [4, 2, 3, 0]
])

print("=== 評価値行列 R ===")
print(R)
print(f"\n行列のサイズ: {R.shape[0]}ユーザー × {R.shape[1]}アイテム")

In [ ]:
# 評価値行列の可視化
fig, ax = plt.subplots(figsize=(8, 6))

# ヒートマップを作成（0は欠損値として特別扱い）
masked_R = np.ma.masked_where(R == 0, R)
im = ax.imshow(masked_R, cmap='YlOrRd', vmin=1, vmax=5)

# 欠損値（0）をグレーで表示
ax.imshow(np.where(R == 0, 1, np.nan), cmap='gray', vmin=0, vmax=1, alpha=0.3)

# 軸ラベル
ax.set_xticks(range(4))
ax.set_yticks(range(4))
ax.set_xticklabels(['item1', 'item2', 'item3', 'item4'])
ax.set_yticklabels(['user1', 'user2', 'user3', 'user4'])

# 各セルに値を表示
for i in range(4):
    for j in range(4):
        if R[i, j] == 0:
            text = ax.text(j, i, '−', ha='center', va='center', color='gray', fontsize=14)
        else:
            text = ax.text(j, i, R[i, j], ha='center', va='center', color='black', fontsize=14)

ax.set_title('評価値行列 R（グレー: 欠損値）')
plt.colorbar(im, label='評価値')
plt.tight_layout()
plt.show()

---
# 2-3 評価値の予測を数理モデルで実現する ー協調フィルタリングと行列因子分解ー

## レコメンドの前提条件

- ユーザーは、購入したすべての商品に評価値を付与している
- 逆に、ユーザーは購入していない商品に評価値を付与していない
- レコメンドする商品は、そのユーザーが未購入の商品（評価値が欠損している商品）に限る

つまり、**欠損部分の評価値を予測値として算出できれば、「もし購入したらいくらの評価を付けるか」を推定できる**。これがレコメンドの有効な判断指標となります。

In [ ]:
# レコメンドの概念図
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 左：評価値がある部分とない部分
ax1 = axes[0]
colors = np.where(R == 0, 0.3, 0.8)
ax1.imshow(colors, cmap='Blues', vmin=0, vmax=1)
for i in range(4):
    for j in range(4):
        if R[i, j] == 0:
            ax1.text(j, i, '?', ha='center', va='center', fontsize=16, color='red', fontweight='bold')
        else:
            ax1.text(j, i, str(R[i, j]), ha='center', va='center', fontsize=14)
ax1.set_xticks(range(4))
ax1.set_yticks(range(4))
ax1.set_xticklabels(['item1', 'item2', 'item3', 'item4'])
ax1.set_yticklabels(['user1', 'user2', 'user3', 'user4'])
ax1.set_title('評価値行列（? = 欠損値 = レコメンド対象）')

# 右：予測値の例
ax2 = axes[1]
R_predicted = R.copy().astype(float)
R_predicted[0, 2] = 4.2  # user1のitem3を予測
R_predicted[1, 2] = 4.5  # user2のitem3を予測
R_predicted[2, 0] = 2.1  # user3のitem1を予測
R_predicted[3, 3] = 3.8  # user4のitem4を予測

im2 = ax2.imshow(R_predicted, cmap='YlOrRd', vmin=1, vmax=5)
for i in range(4):
    for j in range(4):
        if R[i, j] == 0:
            ax2.text(j, i, f'{R_predicted[i,j]:.1f}', ha='center', va='center', 
                    fontsize=12, color='blue', fontweight='bold')
        else:
            ax2.text(j, i, str(int(R_predicted[i, j])), ha='center', va='center', fontsize=14)
ax2.set_xticks(range(4))
ax2.set_yticks(range(4))
ax2.set_xticklabels(['item1', 'item2', 'item3', 'item4'])
ax2.set_yticklabels(['user1', 'user2', 'user3', 'user4'])
ax2.set_title('予測後（青字 = 予測値）')
plt.colorbar(im2, ax=ax2, label='評価値')

plt.tight_layout()
plt.show()

print("例：user1のitem3の予測値が4.2なら、item3をuser1にレコメンドすべき！")

## 予測値の推計方法：3つのアプローチ

| 手法 | 説明 | 軸 |
|:---|:---|:---|
| ①userを軸とした予測値の算出 | 各itemの評価が似ているuserを選び、そのuserの評価値を用いて予測値を算出 | user |
| ②itemを軸とした予測値の算出 | 各userの評価が似ているitemを選び、そのitemの評価値を用いて予測値を算出 | item |
| ③行列因子分解（MF） | 評価値行列を分解した潜在因子行列を解析的に求めることで、予測値を算出 | − |

①と②は**協調フィルタリング**という手法に分類されます。
③は協調フィルタリングとは異なる手法で、評価値行列を**潜在因子行列**と呼ばれる行列に分解して欠損値を推定するアプローチです。

In [ ]:
# 3つのアプローチの概念図
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# ①userを軸とした予測
ax1 = axes[0]
ax1.text(0.5, 0.9, '①userを軸とした予測値の算出', ha='center', fontsize=11, fontweight='bold', transform=ax1.transAxes)
ax1.text(0.5, 0.7, '似ているユーザーの評価値を利用', ha='center', fontsize=10, transform=ax1.transAxes)
ax1.annotate('', xy=(0.7, 0.4), xytext=(0.3, 0.4),
            arrowprops=dict(arrowstyle='->', color='blue', lw=2))
ax1.text(0.2, 0.4, 'user1', fontsize=10, ha='center', va='center')
ax1.text(0.8, 0.4, 'user2\n(類似)', fontsize=10, ha='center', va='center', color='blue')
ax1.text(0.5, 0.2, '→ user2の評価を参考にuser1の欠損を予測', ha='center', fontsize=9, transform=ax1.transAxes)
ax1.axis('off')

# ②itemを軸とした予測
ax2 = axes[1]
ax2.text(0.5, 0.9, '②itemを軸とした予測値の算出', ha='center', fontsize=11, fontweight='bold', transform=ax2.transAxes)
ax2.text(0.5, 0.7, '似ているアイテムの評価値を利用', ha='center', fontsize=10, transform=ax2.transAxes)
ax2.annotate('', xy=(0.7, 0.4), xytext=(0.3, 0.4),
            arrowprops=dict(arrowstyle='->', color='green', lw=2))
ax2.text(0.2, 0.4, 'item1', fontsize=10, ha='center', va='center')
ax2.text(0.8, 0.4, 'item2\n(類似)', fontsize=10, ha='center', va='center', color='green')
ax2.text(0.5, 0.2, '→ item2の評価を参考にitem1の欠損を予測', ha='center', fontsize=9, transform=ax2.transAxes)
ax2.axis('off')

# ③行列因子分解
ax3 = axes[2]
ax3.text(0.5, 0.9, '③行列因子分解（MF）', ha='center', fontsize=11, fontweight='bold', transform=ax3.transAxes)
ax3.text(0.5, 0.7, 'R ≈ P × Q', ha='center', fontsize=12, transform=ax3.transAxes)
ax3.text(0.15, 0.4, 'R', fontsize=14, ha='center', bbox=dict(boxstyle='round', facecolor='lightblue'))
ax3.text(0.32, 0.4, '≈', fontsize=14, ha='center')
ax3.text(0.5, 0.4, 'P', fontsize=14, ha='center', bbox=dict(boxstyle='round', facecolor='lightgreen'))
ax3.text(0.62, 0.4, '×', fontsize=14, ha='center')
ax3.text(0.75, 0.4, 'Q', fontsize=14, ha='center', bbox=dict(boxstyle='round', facecolor='lightyellow'))
ax3.text(0.5, 0.2, '→ 潜在因子行列に分解して欠損を推定', ha='center', fontsize=9, transform=ax3.transAxes)
ax3.axis('off')

plt.tight_layout()
plt.show()

---
# 2-4 ユーザー同士の類似度で予測値を推計する ー内積の定理とコサイン類似度ー

まずは「①userを軸とした予測値の算出」について解説します。

この手法では、各itemの評価が似ているuserを選び、そのuserの評価値を用いて予測値を算出します。
このとき、「各itemの評価が似ているuserを選ぶ」という処理を実現するために、**コサイン類似度**という手法を採用します。

## 基礎となる数理的手法と情報工学的アプローチ

| 基礎となる数理的手法 | 情報工学的アプローチ |
|:---|:---|
| 内積の定理：$\vec{a} \cdot \vec{b} = |\vec{a}||\vec{b}|\cos\theta$ | コサイン類似度：$\cos(a, b) = \frac{\sum_{k=1}^{n} a_k b_k}{\sqrt{\sum_{k=1}^{n} a_k^2} \sqrt{\sum_{k=1}^{n} b_k^2}}$ |
| ベクトルの大きさ（三平方の定理）：$|\vec{a}| = \sqrt{a_1^2 + a_2^2 + \cdots + a_n^2}$ | |

## 座標空間での表現

例として、userが6人、itemが2個だけの状況を想定し、以下の評価値行列が作成されたとしましょう。

$$R = \begin{pmatrix} 2 & 3 \\ 2 & 5 \\ 0 & 3 \\ 4 & 2 \\ 4 & 4 \\ 3 & 0 \end{pmatrix}$$

この評価値行列を**座標空間**で示すと、ユーザー同士の類似度合いが視覚的に把握できます。

In [ ]:
# 6人のユーザー、2つのアイテム
R_simple = np.array([
    [2, 3],  # user1
    [2, 5],  # user2
    [0, 3],  # user3 (item1未評価)
    [4, 2],  # user4
    [4, 4],  # user5
    [3, 0]   # user6 (item2未評価)
])

print("評価値行列 R:")
print(R_simple)

In [ ]:
# 図2.10: 座標空間でのプロット
fig, ax = plt.subplots(figsize=(8, 8))

users = ['user1', 'user2', 'user3', 'user4', 'user5', 'user6']
colors = ['blue', 'blue', 'gray', 'blue', 'blue', 'gray']

for i, (user, color) in enumerate(zip(users, colors)):
    x, y = R_simple[i]
    if x == 0 or y == 0:
        ax.scatter(x, y, s=100, c='gray', alpha=0.5, zorder=5)
        ax.annotate(user, (x, y), xytext=(5, 5), textcoords='offset points', fontsize=10, color='gray')
    else:
        ax.scatter(x, y, s=100, c='blue', zorder=5)
        ax.annotate(user, (x, y), xytext=(5, 5), textcoords='offset points', fontsize=10)

ax.set_xlim(-0.5, 5.5)
ax.set_ylim(-0.5, 5.5)
ax.set_xlabel('item1への評価値', fontsize=12)
ax.set_ylabel('item2への評価値', fontsize=12)
ax.set_title('図2.10: 座標空間でのユーザー分布', fontsize=14)
ax.grid(True, alpha=0.3)
ax.set_aspect('equal')
ax.axhline(y=0, color='gray', linestyle='--', alpha=0.5)
ax.axvline(x=0, color='gray', linestyle='--', alpha=0.5)
ax.text(-0.3, 0, '評価\nなし', fontsize=9, va='center', color='gray')
ax.text(0, -0.3, '評価なし', fontsize=9, ha='center', color='gray')

plt.tight_layout()
plt.show()

## ベクトルによる表現

ここで、目的を「user1に似ているユーザーが誰かを特定すること」とし、コサイン類似度によって類似ユーザーを特定します。

考えやすいように、考察対象をuser1, user2, user4に絞り、各userをベクトルで表現します。

$$\vec{u}_1 = (2, 3), \quad \vec{u}_2 = (2, 5), \quad \vec{u}_4 = (4, 2)$$

In [ ]:
# 図2.11: ベクトルとしての表現
fig, ax = plt.subplots(figsize=(8, 8))

u1 = np.array([2, 3])
u2 = np.array([2, 5])
u4 = np.array([4, 2])

ax.quiver(0, 0, u1[0], u1[1], angles='xy', scale_units='xy', scale=1, 
          color='blue', width=0.02, label=f'$\\vec{{u}}_1$ = (2, 3)')
ax.quiver(0, 0, u2[0], u2[1], angles='xy', scale_units='xy', scale=1, 
          color='green', width=0.02, label=f'$\\vec{{u}}_2$ = (2, 5)')
ax.quiver(0, 0, u4[0], u4[1], angles='xy', scale_units='xy', scale=1, 
          color='red', width=0.02, label=f'$\\vec{{u}}_4$ = (4, 2)')

ax.plot(u1[0], u1[1], 'bo', markersize=8)
ax.plot(u2[0], u2[1], 'go', markersize=8)
ax.plot(u4[0], u4[1], 'ro', markersize=8)

ax.annotate(f'({u1[0]},{u1[1]})\nuser1', (u1[0], u1[1]), xytext=(10, 5), textcoords='offset points', fontsize=10)
ax.annotate(f'({u2[0]},{u2[1]})\nuser2', (u2[0], u2[1]), xytext=(10, 5), textcoords='offset points', fontsize=10)
ax.annotate(f'({u4[0]},{u4[1]})\nuser4', (u4[0], u4[1]), xytext=(10, -15), textcoords='offset points', fontsize=10)

ax.set_xlim(-0.5, 5.5)
ax.set_ylim(-0.5, 5.5)
ax.set_xlabel('item1', fontsize=12)
ax.set_ylabel('item2', fontsize=12)
ax.set_title('図2.11: user1, user2, user4をベクトルで表現', fontsize=14)
ax.grid(True, alpha=0.3)
ax.set_aspect('equal')
ax.legend(loc='lower right')

plt.tight_layout()
plt.show()

print("図から、user1に似ているのはuser2だと言えそうです。")
print("なぜなら、ベクトルを示す矢印のuser1とuser2が作る角度がuser1とuser4が作る角度よりも小さいからです。")

## 内積の定理

角度を「目」で見て類似度を評価することはできますが、コンピュータは「目」で判断することはできず、**数値**でしか判断できません。

そこで、この角度を**定量化**します。このときに便利な数理的手法が高校数学で学習した**内積の定理**です。

2つのベクトル $\vec{a}$, $\vec{b}$ があり、$\vec{a}$, $\vec{b}$ のなす角を $\theta$（theta：シータ）とします。

すると、**内積**は式 (2-3) のように定義されます。

$$\vec{a} \cdot \vec{b} = |\vec{a}||\vec{b}|\cos\theta \quad (2\text{-}3)$$

## cos θ の計算

$$\cos\theta = \frac{\vec{a} \cdot \vec{b}}{|\vec{a}||\vec{b}|} \quad (2\text{-}4)$$

### 分子：内積 $\vec{a} \cdot \vec{b}$ の計算

$$\vec{a} \cdot \vec{b} = a_1 b_1 + a_2 b_2 \quad (2\text{-}6)$$

### 分母：ベクトルの大きさ（ノルム）の計算

$$|\vec{a}| = \sqrt{a_1^2 + a_2^2} \quad (2\text{-}10)$$

In [ ]:
# コサイン類似度の計算（基本版）
def cosine_similarity(a, b):
    """
    2つのベクトルのコサイン類似度を計算する
    式: cos(a, b) = (a · b) / (|a| × |b|)
    """
    a = np.array(a)
    b = np.array(b)
    
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    
    if norm_a == 0 or norm_b == 0:
        return 0.0
    
    return dot_product / (norm_a * norm_b)

print("cosine_similarity関数を定義しました")

In [ ]:
# user1, user2, user4のコサイン類似度を計算
u1 = np.array([2, 3])
u2 = np.array([2, 5])
u4 = np.array([4, 2])

print("=== コサイン類似度の計算 ===")
print(f"u1 = {tuple(u1)}")
print(f"u2 = {tuple(u2)}")
print(f"u4 = {tuple(u4)}")
print()

cos_12 = cosine_similarity(u1, u2)
cos_14 = cosine_similarity(u1, u4)

print(f"cos(u1, u2) = {cos_12:.4f}")
print(f"cos(u1, u4) = {cos_14:.4f}")
print()
print(f"結論: cos(u1, u2) = {cos_12:.3f} > cos(u1, u4) = {cos_14:.3f}")
print("→ user2のほうがuser1に似ている")

---
# 2-5 コサイン類似度の意味を考える ー三角関数ー

## 単位円を用いた cos θ の定義

**単位円**とは原点を中心とした半径1の円です。

- 単位円上に点 $P$ を取り、原点 $O$ を中心に $x$ 軸の正の部分から反時計回りに $\theta$ だけ回転させた線分 $OP$ について
- $P$ の $x$ 座標を $\cos\theta$ と定義します
- $P$ の $y$ 座標は $\sin\theta$ と定義します

## θ と cos θ の関係

$0 \leq \theta \leq 180°$ のとき、$\theta$ の値が大きくなるほど $\cos\theta$ の値が小さくなります。

$$-1 \leq \frac{\vec{a} \cdot \vec{b}}{|\vec{a}||\vec{b}|} \leq 1 \quad (2\text{-}7)$$

だから、計算結果が**1に近いほど**、2つのベクトルは「**似ている**」と言えるのです。

In [ ]:
# θ と cos θ の関係グラフ
fig, ax = plt.subplots(figsize=(10, 5))

theta_deg = np.linspace(0, 180, 100)
theta_rad = np.radians(theta_deg)
cos_values = np.cos(theta_rad)

ax.plot(theta_deg, cos_values, 'b-', linewidth=2)

important_points = [(0, 1, '同じ方向\n(最も類似)'), 
                    (90, 0, '直交\n(無関係)'), 
                    (180, -1, '反対方向\n(最も非類似)')]

for angle, cos_val, label in important_points:
    ax.plot(angle, cos_val, 'ro', markersize=10)
    ax.annotate(f'{label}\n({angle}°, {cos_val})', xy=(angle, cos_val),
                xytext=(angle+15, cos_val), fontsize=10,
                arrowprops=dict(arrowstyle='->', color='gray'))

ax.axhline(y=0, color='k', linewidth=0.5, linestyle='--')
ax.fill_between(theta_deg, 0, cos_values, where=(cos_values > 0), alpha=0.3, color='green', label='類似 (cos θ > 0)')
ax.fill_between(theta_deg, 0, cos_values, where=(cos_values < 0), alpha=0.3, color='red', label='非類似 (cos θ < 0)')

ax.set_xlabel('角度 θ (度)', fontsize=12)
ax.set_ylabel('cos θ', fontsize=12)
ax.set_title('角度θとcos θの関係', fontsize=14)
ax.set_xticks([0, 30, 60, 90, 120, 150, 180])
ax.set_ylim(-1.2, 1.2)
ax.grid(True, alpha=0.3)
ax.legend(loc='upper right')

plt.tight_layout()
plt.show()

---
# 2-6 コサイン類似度を複数のアイテムに適用する ー多次元への拡張ー

## n次元への拡張

アイテム数が $n$ 個になった場合：

$$\vec{a} \cdot \vec{b} = \sum_{k=1}^{n} a_k b_k \quad (2\text{-}9)$$

$$|\vec{a}| = \sqrt{\sum_{k=1}^{n} a_k^2} \quad (2\text{-}12)$$

## 一般化されたコサイン類似度の公式

$$\cos(a, b) = \frac{\vec{a} \cdot \vec{b}}{|\vec{a}||\vec{b}|} = \frac{\sum_{k=1}^{n} a_k b_k}{\sqrt{\sum_{k=1}^{n} a_k^2} \sqrt{\sum_{k=1}^{n} b_k^2}} \quad (2\text{-}13)$$

---
# 2-7 コサイン類似度を改良する ー中心化ー

## 問題点：評価が「逆」のユーザー

例えば、item1〜5について評価が「逆」になっている2人のuserの評価値ベクトルを考えます。

$$\vec{u}_A = (1, 2, 3, 4, 5), \quad \vec{u}_B = (5, 4, 3, 2, 1)$$

両者の評価は真逆なので、コサイン類似度は低く算出されて欲しいところです。

In [ ]:
# 評価が「逆」のユーザーでコサイン類似度を計算
u_A = np.array([1, 2, 3, 4, 5])
u_B = np.array([5, 4, 3, 2, 1])

print("=== 評価が「逆」のユーザー ===")
print(f"u_A = {u_A}")
print(f"u_B = {u_B}")
print()

# 内積の計算
dot_AB = np.dot(u_A, u_B)
print(f"内積: u_A · u_B = 1×5 + 2×4 + 3×3 + 4×2 + 5×1 = {dot_AB}")

# ノルムの計算
norm_A = np.linalg.norm(u_A)
norm_B = np.linalg.norm(u_B)
print(f"|u_A| = √(1² + 2² + 3² + 4² + 5²) = √{sum(u_A**2)} = {norm_A:.4f}")
print(f"|u_B| = √(5² + 4² + 3² + 2² + 1²) = √{sum(u_B**2)} = {norm_B:.4f}")

# コサイン類似度
cos_AB = cosine_similarity(u_A, u_B)
print(f"\ncos(u_A, u_B) = {dot_AB} / ({norm_A:.4f} × {norm_B:.4f}) = {cos_AB:.3f}")
print()
print("問題：評価が真逆なのに、コサイン類似度が0.636と高い！")
print("これは最大値1に近く、「似ている」と判断されてしまう。")

## 解決策：中心化（Centering）

この評価値の差を緩和します。具体的には、userごとに各アイテムへの評価値の**平均値**を算出し、その平均値を各評価値から引きます。

この処理を**中心化**と言います。

In [ ]:
# 図2.21: 中心化の可視化
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

items = ['item1', 'item2', 'item3', 'item4', 'item5']

# 左：中心化前
ax1 = axes[0]
x = np.arange(len(items))
width = 0.35
ax1.bar(x - width/2, u_A, width, label='user A', color='blue', alpha=0.7)
ax1.bar(x + width/2, u_B, width, label='user B', color='orange', alpha=0.7)
ax1.set_xticks(x)
ax1.set_xticklabels(items)
ax1.set_ylabel('評価値')
ax1.set_title('中心化前')
ax1.legend()
ax1.axhline(y=np.mean(u_A), color='blue', linestyle='--', alpha=0.5, label=f'A平均={np.mean(u_A)}')
ax1.axhline(y=np.mean(u_B), color='orange', linestyle='--', alpha=0.5, label=f'B平均={np.mean(u_B)}')
ax1.set_ylim(0, 6)

# 中心化
u_A_centered = u_A - np.mean(u_A)
u_B_centered = u_B - np.mean(u_B)

# 右：中心化後
ax2 = axes[1]
ax2.bar(x - width/2, u_A_centered, width, label='user A (中心化後)', color='blue', alpha=0.7)
ax2.bar(x + width/2, u_B_centered, width, label='user B (中心化後)', color='orange', alpha=0.7)
ax2.set_xticks(x)
ax2.set_xticklabels(items)
ax2.set_ylabel('評価値（中心化後）')
ax2.set_title('中心化後')
ax2.legend()
ax2.axhline(y=0, color='black', linestyle='-', alpha=0.5)
ax2.set_ylim(-3, 3)

plt.suptitle('図2.21: ユーザーごとに評価値の平均値を算出し、各評価値から減算する', fontsize=12)
plt.tight_layout()
plt.show()

print(f"u_A 平均値: {np.mean(u_A)}")
print(f"u_A 中心化後: {u_A_centered}")
print(f"u_B 平均値: {np.mean(u_B)}")
print(f"u_B 中心化後: {u_B_centered}")

In [ ]:
# 中心化後のコサイン類似度を計算
print("=== 中心化後のコサイン類似度計算 ===")
print(f"u_A (中心化後) = {u_A_centered}")
print(f"u_B (中心化後) = {u_B_centered}")
print()

# 内積
dot_AB_centered = np.dot(u_A_centered, u_B_centered)
print(f"内積: {u_A_centered[0]:.0f}×{u_B_centered[0]:.0f} + {u_A_centered[1]:.0f}×{u_B_centered[1]:.0f} + ... = {dot_AB_centered}")

# ノルム
norm_A_centered = np.linalg.norm(u_A_centered)
norm_B_centered = np.linalg.norm(u_B_centered)
print(f"|u_A| = √{sum(u_A_centered**2):.0f} = {norm_A_centered:.4f}")
print(f"|u_B| = √{sum(u_B_centered**2):.0f} = {norm_B_centered:.4f}")

# コサイン類似度
cos_AB_centered = cosine_similarity(u_A_centered, u_B_centered)
print(f"\ncos(u_A, u_B) = {dot_AB_centered} / ({norm_A_centered:.4f} × {norm_B_centered:.4f}) = {cos_AB_centered:.3f}")
print()
print("結果：コサイン類似度が -1 になりました！")
print("これは最小値で、「評価が真逆」という判断と一致します。")

## 改良したコサイン類似度の公式

各要素から平均値を引いた計算が実行されればよいので、改良したモデルは式 (2-14) のようになります。

$\bar{a}$, $\bar{b}$ は、それぞれ $\vec{a}$, $\vec{b}$ の平均評価値を表しています。

$$\cos(a, b) = \frac{\sum_{k=1}^{n} (a_k - \bar{a})(b_k - \bar{b})}{\sqrt{\sum_{k=1}^{n} (a_k - \bar{a})^2} \sqrt{\sum_{k=1}^{n} (b_k - \bar{b})^2}} \quad (2\text{-}14)$$

In [ ]:
# 中心化付きコサイン類似度関数
def cosine_similarity_centered(a, b, mask_a=None, mask_b=None):
    """
    中心化を行ったコサイン類似度を計算する
    
    Parameters:
    -----------
    a, b : array-like
        評価ベクトル
    mask_a, mask_b : array-like, optional
        評価が存在する要素を示すマスク（Trueの要素のみ平均計算に使用）
    
    Returns:
    --------
    float
        中心化されたコサイン類似度
    """
    a = np.array(a, dtype=float)
    b = np.array(b, dtype=float)
    
    # マスクが指定されていない場合は、0以外を有効とする
    if mask_a is None:
        mask_a = (a != 0)
    if mask_b is None:
        mask_b = (b != 0)
    
    # 共通して評価しているアイテムのみ使用
    common_mask = mask_a & mask_b
    
    if np.sum(common_mask) == 0:
        return 0.0
    
    # 平均値を計算（共通アイテムのみ）
    mean_a = np.mean(a[common_mask])
    mean_b = np.mean(b[common_mask])
    
    # 中心化
    a_centered = a[common_mask] - mean_a
    b_centered = b[common_mask] - mean_b
    
    # コサイン類似度を計算
    dot_product = np.dot(a_centered, b_centered)
    norm_a = np.linalg.norm(a_centered)
    norm_b = np.linalg.norm(b_centered)
    
    if norm_a == 0 or norm_b == 0:
        return 0.0
    
    return dot_product / (norm_a * norm_b)

print("cosine_similarity_centered関数を定義しました")

---
# 2-8 コサイン類似度を計算する ー指示関数ー

## 指示関数 $\delta_{u,i}$

「評価値が与えられている要素のみを計算対象とする」という制約を与えています。しかし、コサイン類似度に用いられている $\sum$ は、都合よく「これは足す」「これは足さない」という処理をしてくれるわけではありません。

このような条件2の操作を実現するために、**指示関数** $\delta_{u,i}$ を式 (2-15) のように定義します。$u$ はユーザーを示す行、$i$ はアイテムを示す列を示しています。

$$\delta_{u,i} = \begin{cases} 1 & (r_{u,i}\text{が評価値行列に含まれている}) \\ 0 & (r_{u,i}\text{が評価値行列で欠損している}) \end{cases} \quad (2\text{-}15)$$

## 指示関数を組み込んだコサイン類似度

$$\cos(u_1, u_2) = \frac{\sum_{i=1}^{4} \delta_{1,i}(r_{1,i} - \bar{u}_1) \delta_{2,i}(r_{2,i} - \bar{u}_2)}{\sqrt{\sum_{i=1}^{4} \delta_{1,i}(r_{1,i} - \bar{u}_1)^2} \sqrt{\sum_{i=1}^{4} \delta_{2,i}(r_{2,i} - \bar{u}_2)^2}}$$

これにより、欠損している部分は計算から除外され、欠損していない要素のみに対して計算が実行されます。

In [ ]:
# 評価値行列R（図2.24）
R = np.array([
    [2, 3, 0, 5],  # user1: item3が欠損
    [2, 5, 0, 5],  # user2: item3が欠損
    [0, 3, 4, 4],  # user3: item1が欠損
    [4, 2, 3, 0]   # user4: item4が欠損
])

print("=== 評価値行列 R ===")
print(R)
print("\n（0は欠損値を表す）")

In [ ]:
# user1とuser2のコサイン類似度を計算（中心化・指示関数付き）
print("=== cos(u1, u2) の計算 ===")
print()

u1 = R[0]  # [2, 3, 0, 5]
u2 = R[1]  # [2, 5, 0, 5]

print(f"user1: {u1}")
print(f"user2: {u2}")
print()

# 条件1: 評価値を付けていないitemは対象外として平均値を計算
# user1とuser2共通で評価しているのはitem1, item2, item4
common_items = (u1 != 0) & (u2 != 0)
print(f"共通して評価しているアイテム: {['item1', 'item2', 'item3', 'item4'][i] for i, v in enumerate(common_items) if v}")
print(f"→ item1, item2, item4 (item3は両者とも欠損)")
print()

# 平均値の計算（共通アイテムのみ）
mean_u1 = np.mean(u1[common_items])
mean_u2 = np.mean(u2[common_items])
print(f"user1の平均評価値: (2 + 3 + 5) / 3 = {mean_u1:.3f}")
print(f"user2の平均評価値: (2 + 5 + 5) / 3 = {mean_u2:.3f}")
print()

# 中心化
u1_centered = u1[common_items] - mean_u1
u2_centered = u2[common_items] - mean_u2
print(f"user1 (中心化後): {u1_centered}")
print(f"user2 (中心化後): {u2_centered}")
print()

# コサイン類似度の計算
cos_u1_u2 = cosine_similarity_centered(u1, u2)
print(f"cos(u1, u2) = {cos_u1_u2:.3f}")

In [ ]:
# 全ユーザー間のコサイン類似度行列を計算（図2.26）
print("=== 図2.26: ユーザー間の類似度の計算結果を表形式で整理 ===")
print()

n_users = R.shape[0]
similarity_matrix = np.zeros((n_users, n_users))

for i in range(n_users):
    for j in range(n_users):
        similarity_matrix[i, j] = cosine_similarity_centered(R[i], R[j])

# 表形式で表示
print("          user1    user2    user3    user4")
for i, row in enumerate(similarity_matrix):
    print(f"user{i+1}  ", end="")
    for val in row:
        print(f"{val:8.3f} ", end="")
    print()

In [ ]:
# 類似度行列のヒートマップ
fig, ax = plt.subplots(figsize=(8, 6))

im = ax.imshow(similarity_matrix, cmap='RdYlGn', vmin=-1, vmax=1)

ax.set_xticks(range(4))
ax.set_yticks(range(4))
ax.set_xticklabels(['user1', 'user2', 'user3', 'user4'])
ax.set_yticklabels(['user1', 'user2', 'user3', 'user4'])

for i in range(4):
    for j in range(4):
        ax.text(j, i, f'{similarity_matrix[i, j]:.3f}', ha='center', va='center', fontsize=11)

ax.set_title('図2.26: ユーザー間のコサイン類似度行列')
plt.colorbar(im, label='コサイン類似度')
plt.tight_layout()
plt.show()

print("対角線は自分自身との類似度なので1.000")
print("user3とuser4の類似度が最も高い（0.894）")

---
# 2-9 欠損値を推計する数理モデルを設計し、計算を実行する

ここまでの計算結果を用いて、これから欠損値の推計を行っていきます。

## 推計対象

推計の対象となる欠損値を $r_{3,1}$（user3のitem1への評価）とします。

図2.26の表の計算結果を踏まえ、user3に似ている順番に他のユーザーを並べると
$$\cos(u_3, u_4) > \cos(u_3, u_1) > \cos(u_3, u_2)$$
となるので、user3に似ているのはuser4、user1ということにしましょう。

In [ ]:
# 図2.27: 欠損値推計の概念図
fig, ax = plt.subplots(figsize=(10, 6))

# 評価値行列を描画
ax.imshow(np.where(R == 0, 0.3, 0.8), cmap='Blues', vmin=0, vmax=1)

for i in range(4):
    for j in range(4):
        if R[i, j] == 0:
            if i == 2 and j == 0:  # user3のitem1（推計対象）
                ax.text(j, i, '?', ha='center', va='center', fontsize=16, color='red', fontweight='bold')
            else:
                ax.text(j, i, '−', ha='center', va='center', fontsize=14, color='gray')
        else:
            ax.text(j, i, str(R[i, j]), ha='center', va='center', fontsize=14)

ax.set_xticks(range(4))
ax.set_yticks(range(4))
ax.set_xticklabels(['item1', 'item2', 'item3', 'item4'])
ax.set_yticklabels(['user1', 'user2', 'user3', 'user4'])

# 矢印で類似ユーザーからの参照を示す
ax.annotate('', xy=(0, 2), xytext=(0, 0),
            arrowprops=dict(arrowstyle='->', color='green', lw=2))
ax.annotate('', xy=(0, 2), xytext=(0, 3),
            arrowprops=dict(arrowstyle='->', color='green', lw=2))

ax.text(4.5, 0, 'user1\ncos=0.614', fontsize=10, va='center')
ax.text(4.5, 3, 'user4\ncos=0.894', fontsize=10, va='center')
ax.text(4.5, 2, '← 類似ユーザーの\n   評価値を利用', fontsize=10, va='center', color='green')

ax.set_title('図2.27: user3のitem1の欠損値を、類似ユーザー（user1, user4）の評価から推計', fontsize=12)
plt.tight_layout()
plt.show()

## 予測値の計算式

user3のitem1に対する評価値 $r_{3,1}$ を計算するには、例えば次の数式に示すような方法があります。ちなみに評価値 $r_{3,1}$ は予測値として算出されるため、予測値を示す記号^（ハット）を用いて $\hat{r}_{3,1}$ と表記します。

$$\hat{r}_{3,1} = \bar{u}_3 + (r_{1,1} - \bar{u}_1) \times \cos(u_3, u_1) + (r_{4,1} - \bar{u}_4) \times \cos(u_3, u_4)$$

この数式は以下の構造になっています：

- **$\bar{u}_3$**：user3の平均評価値（ベース）
- **$(r_{1,1} - \bar{u}_1)$**：user1のitem1評価と平均との差
- **$\cos(u_3, u_1)$**：user3との類似度合い（重み付け）

In [ ]:
# 図2.28: 欠損値の推計で用いる数値を表形式で整理
print("=== 図2.28: 欠損値の推計で用いる数値 ===")
print()

# user1, user3, user4の情報
users_info = {
    'user1': {'ratings': R[0], 'item1': R[0, 0], 'mean': None, 'cos_with_u3': similarity_matrix[2, 0]},
    'user3': {'ratings': R[2], 'item1': '?', 'mean': None, 'cos_with_u3': '-'},
    'user4': {'ratings': R[3], 'item1': R[3, 0], 'mean': None, 'cos_with_u3': similarity_matrix[2, 3]}
}

# 平均値を計算（共通アイテムでの平均ではなく、各ユーザーの評価平均）
for user, info in users_info.items():
    ratings = info['ratings']
    valid_ratings = ratings[ratings != 0]
    info['mean'] = np.mean(valid_ratings)

print(f"{'':10} {'item1':>8} {'平均評価値':>12} {'user3とのcos':>14}")
print("-" * 50)
for user, info in users_info.items():
    item1_val = info['item1'] if isinstance(info['item1'], str) else f"{info['item1']}"
    cos_val = info['cos_with_u3'] if isinstance(info['cos_with_u3'], str) else f"{info['cos_with_u3']:.3f}"
    print(f"{user:10} {item1_val:>8} {info['mean']:>12.3f} {cos_val:>14}")

In [ ]:
# 予測値を計算
print("=== r̂_{3,1} の計算 ===")
print()

# 各値を取得
mean_u1 = users_info['user1']['mean']
mean_u3 = users_info['user3']['mean']
mean_u4 = users_info['user4']['mean']

r_1_1 = R[0, 0]  # user1のitem1評価 = 2
r_4_1 = R[3, 0]  # user4のitem1評価 = 4

cos_u3_u1 = similarity_matrix[2, 0]  # 0.614
cos_u3_u4 = similarity_matrix[2, 3]  # 0.894

print(f"各パラメータ:")
print(f"  ū₃ = {mean_u3:.3f}")
print(f"  r₁,₁ = {r_1_1}, ū₁ = {mean_u1:.3f}, (r₁,₁ - ū₁) = {r_1_1 - mean_u1:.3f}")
print(f"  r₄,₁ = {r_4_1}, ū₄ = {mean_u4:.3f}, (r₄,₁ - ū₄) = {r_4_1 - mean_u4:.3f}")
print(f"  cos(u₃, u₁) = {cos_u3_u1:.3f}")
print(f"  cos(u₃, u₄) = {cos_u3_u4:.3f}")
print()

# 予測値を計算
r_hat_3_1 = mean_u3 + (r_1_1 - mean_u1) * cos_u3_u1 + (r_4_1 - mean_u4) * cos_u3_u4

print(f"計算式:")
print(f"r̂₃,₁ = ū₃ + (r₁,₁ - ū₁) × cos(u₃, u₁) + (r₄,₁ - ū₄) × cos(u₃, u₄)")
print()
print(f"     = {mean_u3:.3f} + ({r_1_1 - mean_u1:.3f}) × {cos_u3_u1:.3f} + ({r_4_1 - mean_u4:.3f}) × {cos_u3_u4:.3f}")
print(f"     = {mean_u3:.3f} + {(r_1_1 - mean_u1) * cos_u3_u1:.3f} + {(r_4_1 - mean_u4) * cos_u3_u4:.3f}")
print(f"     = {r_hat_3_1:.3f}")
print()
print(f"予測結果: user3のitem1への評価値は {r_hat_3_1:.3f} と推計されました。")

In [ ]:
# 予測結果の解釈
print("=== 予測結果の解釈 ===")
print()
print(f"計算結果は {r_hat_3_1:.3f} と出ました。")
print()
print("user3はitem2に3、item3に4、item4に4の評価値を付与しているので、")
print(f"user3にとって {r_hat_3_1:.3f} という値は平均的な値とも考えられそうです。")
print()
print("であれば、そこまで積極的にitem1をレコメンドする必要はなさそうですし、")
print("レコメンドしたとしても購入される確率はそこまで高くなさそうだ、と推察されます。")
print()
print("ただし、この計算方法はあくまで一例であり、必ずしもこの通りに評価値を推定する")
print("必要はありません。どのように設計すれば目的にかなう数理モデルとなるか、常に")
print("考察と検証を繰り返すことが極めて重要です。")

---
# まとめ

## 本章で学んだこと

### 評価値行列
- ユーザーの評価をベクトルとして表現
- 複数ユーザーの評価を行列 $R$ としてまとめる
- 欠損値（0）= レコメンド対象

### 協調フィルタリング
- 類似ユーザーの評価を使って欠損値を予測
- 「類似性」をコサイン類似度で定量化

### コサイン類似度
$$\cos(a, b) = \frac{\vec{a} \cdot \vec{b}}{|\vec{a}||\vec{b}|} = \frac{\sum_{k=1}^{n} a_k b_k}{\sqrt{\sum_{k=1}^{n} a_k^2} \sqrt{\sum_{k=1}^{n} b_k^2}}$$

### 中心化（改良版）
$$\cos(a, b) = \frac{\sum_{k=1}^{n} (a_k - \bar{a})(b_k - \bar{b})}{\sqrt{\sum_{k=1}^{n} (a_k - \bar{a})^2} \sqrt{\sum_{k=1}^{n} (b_k - \bar{b})^2}}$$

### 指示関数
$$\delta_{u,i} = \begin{cases} 1 & (r_{u,i}\text{が評価値行列に含まれている}) \\ 0 & (r_{u,i}\text{が評価値行列で欠損している}) \end{cases}$$

### 欠損値の予測
$$\hat{r}_{3,1} = \bar{u}_3 + \sum_{u \in \text{similar users}} (r_{u,1} - \bar{u}) \times \cos(u_3, u)$$

In [ ]:
# 最終確認：学んだ関数のまとめ
print("=== 本Notebookで実装した関数 ===")
print()
print("1. cosine_similarity(a, b)")
print("   - 基本的なコサイン類似度を計算")
print()
print("2. cosine_similarity_centered(a, b)")
print("   - 中心化を行ったコサイン類似度を計算")
print("   - 欠損値（0）を自動的に除外")
print()
print("=== 使用例 ===")
print(f"基本版: cosine_similarity([2,3,5], [2,5,5]) = {cosine_similarity([2,3,5], [2,5,5]):.4f}")
print(f"中心化版: cosine_similarity_centered([2,3,0,5], [2,5,0,5]) = {cosine_similarity_centered([2,3,0,5], [2,5,0,5]):.4f}")

---
# 2-10 アイテム同士の類似度で予測値を推計する

前節まではuserを軸とした予測値の算出について解説しました。一方、類似度の計算方法としてはitem同士の類似度を用いることも可能です。

## userベースとitemベースの比較

| 手法 | 説明 |
|:---|:---|
| ①userを軸とした予測値の算出 | 似ているユーザーの評価値を利用 |
| ②itemを軸とした予測値の算出 | 似ているアイテムの評価値を利用 |

実際にECサイトで協調フィルタリングによって欠損値を推定する際は、**user軸よりもitem軸で予測値を算出することが実務上多い**ように見受けられます。

- ECサイトで扱うitem数と訪問してくるuser数を比較すると、user数のほうが多いことが通常
- user同士の類似度よりitem同士の類似度のほうが計算負荷が低くなりやすい

In [ ]:
# 図2.30: アイテム間の類似度行列
print("=== 図2.30: 各item同士の類似度は表形式で整理できる ===")
print()
print("          item1    item2    item3    item4")
print("item1   cos(i₁,i₁) cos(i₁,i₂) cos(i₁,i₃) cos(i₁,i₄)")
print("item2   cos(i₂,i₁) cos(i₂,i₂) cos(i₂,i₃) cos(i₂,i₄)")
print("item3   cos(i₃,i₁) cos(i₃,i₂) cos(i₃,i₃) cos(i₃,i₄)")
print("item4   cos(i₄,i₁) cos(i₄,i₂) cos(i₄,i₃) cos(i₄,i₄)")
print()
print("アイテムベースでは、評価値行列Rの「列ベクトル」を使って類似度を計算します。")

In [ ]:
# 評価値行列R（再掲）
R = np.array([
    [2, 3, 0, 5],
    [2, 5, 0, 5],
    [0, 3, 4, 4],
    [4, 2, 3, 0]
])

print("評価値行列 R:")
print(R)
print()

# 列ベクトルとして各アイテムを表現
i1 = R[:, 0]  # item1の列ベクトル
i2 = R[:, 1]  # item2の列ベクトル
i3 = R[:, 2]  # item3の列ベクトル
i4 = R[:, 3]  # item4の列ベクトル

print("各アイテムの列ベクトル:")
print(f"i₁ = {i1}")
print(f"i₂ = {i2}")
print(f"i₃ = {i3}")
print(f"i₄ = {i4}")

## アイテムベースのコサイン類似度関数

アイテムベースでも同様に、指示関数を用いて欠損値を除外し、中心化を行います。

$$\cos(i_1, i_4) = \frac{\sum_{u=1}^{4} \delta_{u,1}(r_{u,1} - \bar{u}_u) \delta_{u,4}(r_{u,4} - \bar{u}_u)}{\sqrt{\sum_{u=1}^{4} \delta_{u,1}(r_{u,1} - \bar{u}_u)^2} \sqrt{\sum_{u=1}^{4} \delta_{u,4}(r_{u,4} - \bar{u}_u)^2}}$$

In [ ]:
# アイテムベースのコサイン類似度関数
def cosine_similarity_item_based(R, item_i, item_j):
    """
    アイテムベースの中心化コサイン類似度を計算する
    
    Parameters:
    -----------
    R : numpy.ndarray
        評価値行列
    item_i, item_j : int
        比較するアイテムのインデックス
    
    Returns:
    --------
    float
        アイテム間のコサイン類似度
    """
    col_i = R[:, item_i]
    col_j = R[:, item_j]
    
    # 両方のアイテムを評価しているユーザーのみ
    common_users = (col_i != 0) & (col_j != 0)
    
    if np.sum(common_users) == 0:
        return 0.0
    
    # 各ユーザーの平均評価値で中心化
    numerator = 0.0
    norm_i_sq = 0.0
    norm_j_sq = 0.0
    
    for u in range(R.shape[0]):
        if common_users[u]:
            # ユーザーuの平均評価値（欠損値除く）
            user_ratings = R[u, :]
            valid_ratings = user_ratings[user_ratings != 0]
            mean_u = np.mean(valid_ratings)
            
            centered_i = col_i[u] - mean_u
            centered_j = col_j[u] - mean_u
            
            numerator += centered_i * centered_j
            norm_i_sq += centered_i ** 2
            norm_j_sq += centered_j ** 2
    
    if norm_i_sq == 0 or norm_j_sq == 0:
        return 0.0
    
    return numerator / (np.sqrt(norm_i_sq) * np.sqrt(norm_j_sq))

print("cosine_similarity_item_based関数を定義しました")

In [ ]:
# cos(i1, i4)を計算
print("=== cos(i₁, i₄) の計算 ===")
print()

# item1とitem4を共通で評価しているのはuser1とuser2
print("item1とitem4を共通で評価しているユーザー: user1, user2")
print("(user3はitem1が欠損、user4はitem4が欠損)")
print()

# user1とuser2の平均評価値
mean_u1 = np.mean([2, 3, 5])  # user1: [2,3,0,5] → 欠損除外
mean_u2 = np.mean([2, 5, 5])  # user2: [2,5,0,5] → 欠損除外
print(f"ū₁ = (2+3+5)/3 = {mean_u1:.3f}")
print(f"ū₂ = (2+5+5)/3 = {mean_u2:.3f}")
print()

# 中心化した値
print("中心化:")
print(f"  user1: (r₁,₁ - ū₁) = 2 - {mean_u1:.3f} = {2 - mean_u1:.3f}")
print(f"         (r₁,₄ - ū₁) = 5 - {mean_u1:.3f} = {5 - mean_u1:.3f}")
print(f"  user2: (r₂,₁ - ū₂) = 2 - {mean_u2:.3f} = {2 - mean_u2:.3f}")
print(f"         (r₂,₄ - ū₂) = 5 - {mean_u2:.3f} = {5 - mean_u2:.3f}")
print()

# コサイン類似度の計算
cos_i1_i4 = cosine_similarity_item_based(R, 0, 3)
print(f"cos(i₁, i₄) = {cos_i1_i4:.3f}")

In [ ]:
# 全アイテム間のコサイン類似度行列を計算
print("=== アイテム間の類似度行列 ===")
print()

n_items = R.shape[1]
item_similarity_matrix = np.zeros((n_items, n_items))

for i in range(n_items):
    for j in range(n_items):
        item_similarity_matrix[i, j] = cosine_similarity_item_based(R, i, j)

# 表形式で表示
print("          item1    item2    item3    item4")
for i, row in enumerate(item_similarity_matrix):
    print(f"item{i+1}  ", end="")
    for val in row:
        print(f"{val:8.3f} ", end="")
    print()

print()
print(f"cos(i₁, i₄) = {item_similarity_matrix[0, 3]:.3f}")
print(f"cos(i₂, i₄) = {item_similarity_matrix[1, 3]:.3f}")
print(f"cos(i₃, i₄) = {item_similarity_matrix[2, 3]:.3f}")
print()
print("→ item4に最も似ているのはitem3（cos=1.000）、次にitem2（cos=0.090）")

## アイテムベースでの欠損値予測

user4のitem4（$r_{4,4}$）の欠損値を、item軸で推計してみましょう。

item4に似ているのはitem2とitem3なので、user4のitem2, item3に対する評価値を参考にします。

In [ ]:
# 図2.31: 類似度の高いアイテムの評価値を用いて欠損値を推計
fig, ax = plt.subplots(figsize=(10, 6))

ax.imshow(np.where(R == 0, 0.3, 0.8), cmap='Blues', vmin=0, vmax=1)

for i in range(4):
    for j in range(4):
        if R[i, j] == 0:
            if i == 3 and j == 3:  # user4のitem4（推計対象）
                ax.text(j, i, '?', ha='center', va='center', fontsize=16, color='red', fontweight='bold')
            else:
                ax.text(j, i, '−', ha='center', va='center', fontsize=14, color='gray')
        else:
            ax.text(j, i, str(R[i, j]), ha='center', va='center', fontsize=14)

ax.set_xticks(range(4))
ax.set_yticks(range(4))
ax.set_xticklabels(['item1', 'item2', 'item3', 'item4'])
ax.set_yticklabels(['user1', 'user2', 'user3', 'user4'])

# 矢印で類似アイテムからの参照を示す
ax.annotate('', xy=(3, 3), xytext=(1, 3),
            arrowprops=dict(arrowstyle='->', color='green', lw=2))
ax.annotate('', xy=(3, 3), xytext=(2, 3),
            arrowprops=dict(arrowstyle='->', color='green', lw=2))

ax.set_title('図2.31: user4のitem4の欠損値を、類似アイテム（item2, item3）の評価から推計', fontsize=12)
plt.tight_layout()
plt.show()

print("user4の評価値: item1=4, item2=2, item3=3, item4=?")
print(f"cos(i₂, i₄) = {item_similarity_matrix[1, 3]:.3f}")
print(f"cos(i₃, i₄) = {item_similarity_matrix[2, 3]:.3f}")

---
# 2-11 ユーザー目線で数理モデルを再考する ーセレンディピティー

ここまでuser軸とitem軸の2つの観点から、評価値を予測するための数理モデルを解説してきました。ここで、今一度レコメンドの数理モデルについて考え直してみましょう。

## セレンディピティとは

ECサイトを考える上で重要な観点の1つに、**セレンディピティ**という概念があります。これは「目新しさ」といった意味合いで使われる概念です。

### item軸の問題点

item軸でのレコメンドを運用していると起きがちな現象：
- ECサイトに訪問するたびに毎回同じような商品ばかりがレコメンドされる
- ユーザーは見飽きてしまい、そのECサイトを利用しなくなることが懸念される
- 似ているアイテムばかりがレコメンドされても、そのユーザーが既に類似商品を持っていたりすると、レコメンドした商品への興味は薄い可能性が高い

In [ ]:
# セレンディピティの概念図
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 左：item軸の問題点
ax1 = axes[0]
ax1.text(0.5, 0.9, 'item軸でのレコメンド', ha='center', fontsize=12, fontweight='bold', transform=ax1.transAxes)
ax1.text(0.5, 0.7, '毎回似たような商品ばかり', ha='center', fontsize=10, color='red', transform=ax1.transAxes)

# 同じような商品のイメージ
for i in range(3):
    ax1.add_patch(plt.Rectangle((0.2 + i*0.2, 0.3), 0.15, 0.2, fill=True, color='lightblue', alpha=0.8))
    ax1.text(0.275 + i*0.2, 0.4, f'商品{i+1}', ha='center', fontsize=9)

ax1.text(0.5, 0.15, '→ ユーザーは「また同じか...」と飽きてしまう', ha='center', fontsize=10, transform=ax1.transAxes)
ax1.axis('off')
ax1.set_title('問題：セレンディピティの欠如')

# 右：user軸の利点
ax2 = axes[1]
ax2.text(0.5, 0.9, 'user軸でのレコメンド', ha='center', fontsize=12, fontweight='bold', transform=ax2.transAxes)
ax2.text(0.5, 0.7, '自分と嗜好が似ているユーザーが\n興味を持っている商品を発見', ha='center', fontsize=10, color='green', transform=ax2.transAxes)

# 異なる商品のイメージ
colors = ['lightblue', 'lightgreen', 'lightyellow']
for i, color in enumerate(colors):
    ax2.add_patch(plt.Rectangle((0.2 + i*0.2, 0.3), 0.15, 0.2, fill=True, color=color, alpha=0.8))
    ax2.text(0.275 + i*0.2, 0.4, f'新発見{i+1}', ha='center', fontsize=9)

ax2.text(0.5, 0.15, '→ UX（User Experience）の向上が期待できる', ha='center', fontsize=10, transform=ax2.transAxes)
ax2.axis('off')
ax2.set_title('解決：意外な商品との出会い')

plt.tight_layout()
plt.show()

print("セレンディピティ = 思いがけない発見、偶然の出会い")
print()
print("user軸でのレコメンドは、自分と嗜好が似ている他のユーザーが興味を持っている")
print("アイテムに基づき欠損値を予測するため、item軸ではレコメンドされなかった")
print("商品に出会える可能性がある。")

## 協調フィルタリングの限界

つまり、item軸とuser軸、どちらのアプローチで欠損値を予測したとしても、セレンディピティという観点では優れたレコメンドを実現できない可能性がある、ということです。

では、セレンディピティという観点から考えると、どのような数理モデルによって評価値を予測すればよいのでしょうか。

この課題を解決するために、次節からより高度な数理モデルを考察します。

---
# 2-12 課題解決のために数理モデルを変更する ー行列因子分解ー

本節以降では、「③行列因子分解（MF: Matrix Factorization）」によって評価値を推定する数理モデルを導出します。その準備として、まずは**残差行列**を導出し、行列の**和**、**差**及び**積**への理解が必要となります。

## 基礎となる数理的手法

### 行列の和
$$A + B = \begin{pmatrix} a_{1,1} & a_{1,2} \\ a_{2,1} & a_{2,2} \end{pmatrix} + \begin{pmatrix} b_{1,1} & b_{1,2} \\ b_{2,1} & b_{2,2} \end{pmatrix} = \begin{pmatrix} a_{1,1}+b_{1,1} & a_{1,2}+b_{1,2} \\ a_{2,1}+b_{2,1} & a_{2,2}+b_{2,2} \end{pmatrix}$$

### 行列の差
$$A - B = \begin{pmatrix} a_{1,1} & a_{1,2} \\ a_{2,1} & a_{2,2} \end{pmatrix} - \begin{pmatrix} b_{1,1} & b_{1,2} \\ b_{2,1} & b_{2,2} \end{pmatrix} = \begin{pmatrix} a_{1,1}-b_{1,1} & a_{1,2}-b_{1,2} \\ a_{2,1}-b_{2,1} & a_{2,2}-b_{2,2} \end{pmatrix}$$

### 行列の積
$$AB = \begin{pmatrix} a_{1,1} & a_{1,2} \\ a_{2,1} & a_{2,2} \end{pmatrix} \begin{pmatrix} b_{1,1} & b_{1,2} \\ b_{2,1} & b_{2,2} \end{pmatrix} = \begin{pmatrix} a_{1,1}b_{1,1}+a_{1,2}b_{2,1} & a_{1,1}b_{1,2}+a_{1,2}b_{2,2} \\ a_{2,1}b_{1,1}+a_{2,2}b_{2,1} & a_{2,1}b_{1,2}+a_{2,2}b_{2,2} \end{pmatrix}$$

In [ ]:
# 行列演算の基礎
print("=== 行列演算の例 ===")
print()

A = np.array([[1, 2], [3, 1]])
B = np.array([[2, 3], [1, 2]])

print("行列A:")
print(A)
print()
print("行列B:")
print(B)
print()

print("--- 行列の和 A + B ---")
print(A + B)
print()

print("--- 行列の差 A - B ---")
print(A - B)
print()

print("--- 行列の積 AB ---")
print(A @ B)
print()
print("計算過程:")
print(f"  (1,1)要素: 1×2 + 2×1 = {1*2 + 2*1}")
print(f"  (1,2)要素: 1×3 + 2×2 = {1*3 + 2*2}")
print(f"  (2,1)要素: 3×2 + 1×1 = {3*2 + 1*1}")
print(f"  (2,2)要素: 3×3 + 1×2 = {3*3 + 1*2}")

## 行列積の規則性

行列同士の積の計算結果には規則性があります。重要なのは行列の**形**が重要だということです。

**規則**: $m \times n$ 行列と $n \times p$ 行列の積は $m \times p$ 行列になる

$$\underbrace{(2 \times 3)}_{A} \times \underbrace{(3 \times 2)}_{B} = \underbrace{(2 \times 2)}_{AB}$$

隣り合う数字が同じ（上の例では3）でないと、行列の積は計算できません。

In [ ]:
# 図2.36: 行列積が可能な場合と不可の場合
fig, axes = plt.subplots(1, 2, figsize=(14, 4))

# 左：計算可能
ax1 = axes[0]
ax1.text(0.1, 0.6, '4×2', fontsize=14, ha='center', bbox=dict(boxstyle='round', facecolor='lightblue'))
ax1.text(0.25, 0.6, '×', fontsize=14, ha='center')
ax1.text(0.4, 0.6, '2×3', fontsize=14, ha='center', bbox=dict(boxstyle='round', facecolor='lightgreen'))
ax1.text(0.55, 0.6, '=', fontsize=14, ha='center')
ax1.text(0.7, 0.6, '4×3', fontsize=14, ha='center', bbox=dict(boxstyle='round', facecolor='lightyellow'))

# 矢印で一致を示す
ax1.annotate('', xy=(0.35, 0.45), xytext=(0.15, 0.45),
            arrowprops=dict(arrowstyle='<->', color='green', lw=2))
ax1.text(0.25, 0.35, '一致!', fontsize=10, ha='center', color='green')

ax1.set_xlim(0, 1)
ax1.set_ylim(0, 1)
ax1.axis('off')
ax1.set_title('計算可能', fontsize=12)

# 右：計算不可
ax2 = axes[1]
ax2.text(0.1, 0.6, '4×2', fontsize=14, ha='center', bbox=dict(boxstyle='round', facecolor='lightblue'))
ax2.text(0.25, 0.6, '×', fontsize=14, ha='center')
ax2.text(0.4, 0.6, '3×2', fontsize=14, ha='center', bbox=dict(boxstyle='round', facecolor='lightcoral'))
ax2.text(0.55, 0.6, '=', fontsize=14, ha='center')
ax2.text(0.7, 0.6, '×', fontsize=20, ha='center', color='red')

# 矢印で不一致を示す
ax2.annotate('', xy=(0.35, 0.45), xytext=(0.15, 0.45),
            arrowprops=dict(arrowstyle='<->', color='red', lw=2))
ax2.text(0.25, 0.35, '不一致!', fontsize=10, ha='center', color='red')

ax2.set_xlim(0, 1)
ax2.set_ylim(0, 1)
ax2.axis('off')
ax2.set_title('計算できない', fontsize=12)

plt.suptitle('図2.36: 行列積が可能な場合と不可の場合', fontsize=12)
plt.tight_layout()
plt.show()

## 残差行列

評価値行列 $R$ と予測値行列 $\hat{R}$ の差を**残差行列** $E$ と呼びます。

$$E = R - \hat{R} = R - PQ$$

ここで：
- $R$: 評価値行列（$U \times I$ 行列）
- $\hat{R}$: 予測値行列（$U \times I$ 行列）
- $P$: ユーザー因子行列（$U \times d$ 行列）
- $Q$: アイテム因子行列（$d \times I$ 行列）
- $d$: **潜在因子**の数

## 潜在因子モデル

評価値行列がスパース（疎）な行列となりがちです。このような評価値行列が抱える実務上の問題に対応するために、数理モデルとして**潜在因子モデル**がしばしば活用されます。

潜在因子モデルでは、ユーザーとアイテムの関係を、いくつかの**潜在因子**に基づいて表現します。潜在因子とは、例えばユーザーの隠れた嗜好やアイテムの隠れた特性を反映しており、具体的な意味内容を持つこともあれば、データから抽出される抽象的な（意味付けることが難しい）特徴であることもあります。

## 行列因子分解（Matrix Factorization）

行列因子分解のアプローチでは、評価値行列 $R$ を**ユーザー因子行列** $P$ と**アイテム因子行列** $Q$ の積に分解します。

$$R \approx P \times Q$$

In [ ]:
# 図2.34: 行列因子分解の概念図
fig, ax = plt.subplots(figsize=(14, 5))

# R (U×I)
ax.add_patch(plt.Rectangle((0.05, 0.3), 0.15, 0.4, fill=True, color='lightblue', alpha=0.8))
ax.text(0.125, 0.5, 'R', fontsize=20, ha='center', va='center', fontweight='bold')
ax.text(0.125, 0.75, 'U×I', fontsize=10, ha='center')
ax.text(0.125, 0.2, '評価値行列', fontsize=9, ha='center')

# ≈
ax.text(0.25, 0.5, '≈', fontsize=24, ha='center', va='center')

# P (U×d)
ax.add_patch(plt.Rectangle((0.32, 0.3), 0.08, 0.4, fill=True, color='lightgreen', alpha=0.8))
ax.text(0.36, 0.5, 'P', fontsize=20, ha='center', va='center', fontweight='bold')
ax.text(0.36, 0.75, 'U×d', fontsize=10, ha='center')
ax.text(0.36, 0.2, 'ユーザー\n因子行列', fontsize=9, ha='center')

# ×
ax.text(0.45, 0.5, '×', fontsize=20, ha='center', va='center')

# Q (d×I)
ax.add_patch(plt.Rectangle((0.52, 0.4), 0.15, 0.15, fill=True, color='lightyellow', alpha=0.8))
ax.text(0.595, 0.475, 'Q', fontsize=20, ha='center', va='center', fontweight='bold')
ax.text(0.595, 0.6, 'd×I', fontsize=10, ha='center')
ax.text(0.595, 0.3, 'アイテム因子行列', fontsize=9, ha='center')

# 説明
ax.text(0.8, 0.6, 'd = 潜在因子の数', fontsize=11, ha='left')
ax.text(0.8, 0.5, 'U = ユーザー数', fontsize=11, ha='left')
ax.text(0.8, 0.4, 'I = アイテム数', fontsize=11, ha='left')

ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
ax.axis('off')
ax.set_title('図2.34: 評価値行列Rをユーザー因子行列Pとアイテム因子行列Qに分解する', fontsize=12)
plt.tight_layout()
plt.show()

print("行列PとQは元の評価値行列Rよりも低ランクの行列となる。")
print("dは潜在因子の数で、ハイパーパラメータとして設定する。")

---
# 2-13 評価値の推計を最適化問題に置き換える ー残差行列と誤差ー

では、具体的な数理モデルの考察に入ります。このケースでもuser軸、item軸で考察した評価値行列を用います。

## 評価値行列の一般化表現

$$R = \begin{pmatrix} r_{1,1} & r_{1,2} & r_{1,3} & r_{1,4} \\ r_{2,1} & r_{2,2} & r_{2,3} & r_{2,4} \\ r_{3,1} & r_{3,2} & r_{3,3} & r_{3,4} \\ r_{4,1} & r_{4,2} & r_{4,3} & r_{4,4} \end{pmatrix} = \begin{pmatrix} 2 & 3 & 0 & 5 \\ 2 & 5 & 0 & 5 \\ 0 & 3 & 4 & 4 \\ 4 & 2 & 3 & 0 \end{pmatrix}$$

## 潜在因子行列の表現

$P$ は $4 \times 2$ 行列、$Q$ は $2 \times 4$ 行列です。

$$P = \begin{pmatrix} p_{1,1} & p_{1,2} \\ p_{2,1} & p_{2,2} \\ p_{3,1} & p_{3,2} \\ p_{4,1} & p_{4,2} \end{pmatrix}, \quad Q = \begin{pmatrix} q_{1,1} & q_{1,2} & q_{1,3} & q_{1,4} \\ q_{2,1} & q_{2,2} & q_{2,3} & q_{2,4} \end{pmatrix}$$

In [ ]:
# P, Qの一般化表現
print("=== 潜在因子行列の設定 ===")
print()

# d=2として設定
d = 2
n_users = 4
n_items = 4

print(f"潜在因子数 d = {d}")
print(f"ユーザー数 U = {n_users}")
print(f"アイテム数 I = {n_items}")
print()

print(f"ユーザー因子行列 P: {n_users}×{d} 行列")
print(f"アイテム因子行列 Q: {d}×{n_items} 行列")
print()

# 行列Pの要素
print("P の要素:")
for u in range(1, n_users+1):
    row = [f"p_{u},{k}" for k in range(1, d+1)]
    print(f"  user{u}: [{', '.join(row)}]")

print()

# 行列Qの要素
print("Q の要素:")
for k in range(1, d+1):
    row = [f"q_{k},{i}" for i in range(1, n_items+1)]
    print(f"  潜在因子{k}: [{', '.join(row)}]")

## 予測値の計算

$PQ$ の積を計算すると、各要素は以下のようになります。

$$\hat{r}_{u,i} = \sum_{k=1}^{d} p_{u,k} \cdot q_{k,i}$$

例えば $\hat{r}_{1,3}$（user1のitem3の予測値）は：

$$\hat{r}_{1,3} = p_{1,1} \cdot q_{1,3} + p_{1,2} \cdot q_{2,3}$$

## 残差（誤差）

評価値と予測値の差（残差）は：

$$e_{u,i} = r_{u,i} - \hat{r}_{u,i} = r_{u,i} - \sum_{k=1}^{d} p_{u,k} \cdot q_{k,i}$$

**行列因子分解の目標は、この残差を最小化するようなPとQを見つけること**です。

In [ ]:
# 残差の計算例
print("=== 残差の計算例 ===")
print()

# 評価値行列
R = np.array([
    [2, 3, 0, 5],
    [2, 5, 0, 5],
    [0, 3, 4, 4],
    [4, 2, 3, 0]
])

# ランダムな初期値でP, Qを設定（実際は最適化で求める）
np.random.seed(42)
P_init = np.random.rand(4, 2)
Q_init = np.random.rand(2, 4)

print("初期のユーザー因子行列 P:")
print(np.round(P_init, 3))
print()

print("初期のアイテム因子行列 Q:")
print(np.round(Q_init, 3))
print()

# 予測行列 R̂ = PQ
R_hat = P_init @ Q_init

print("予測行列 R̂ = PQ:")
print(np.round(R_hat, 3))
print()

# 残差行列 E = R - R̂（欠損値は除く）
E = R - R_hat
print("残差行列 E = R - R̂:")
print(np.round(E, 3))
print()

# 欠損値を除いた二乗誤差の合計
mask = (R != 0)
squared_error = np.sum((E[mask]) ** 2)
print(f"二乗誤差の合計（欠損値除く）: {squared_error:.3f}")
print()
print("この二乗誤差を最小化するようにP, Qを更新していきます。")
print("これが「最適化問題」としての行列因子分解です。")

---
# まとめ（Part 2）

## 2-10〜2-13で学んだこと

### アイテムベース協調フィルタリング
- 列ベクトルでアイテムを表現
- アイテム間のコサイン類似度を計算
- 類似アイテムの評価値から欠損値を予測

### セレンディピティ
- レコメンドにおける「目新しさ」の重要性
- item軸だけでは似た商品ばかりになりがち
- ユーザー体験（UX）の観点からの課題

### 行列因子分解（Matrix Factorization）
$$R \approx P \times Q$$

- $R$: 評価値行列（$U \times I$）
- $P$: ユーザー因子行列（$U \times d$）
- $Q$: アイテム因子行列（$d \times I$）
- $d$: 潜在因子の数（ハイパーパラメータ）

### 残差と最適化
$$e_{u,i} = r_{u,i} - \sum_{k=1}^{d} p_{u,k} \cdot q_{k,i}$$

目標：残差（二乗誤差）を最小化する $P$, $Q$ を見つける

## 次のステップ（教科書の続き）
- 損失関数の定義
- 勾配降下法による最適化
- 正則化項の追加

---
# 2-14 最適化問題を解く ー損失関数ー

この最適化問題を解く手法にはさまざまなパターンがありますが、本章では基本的かつ根本的な考え方を学べる**勾配降下法**を採用して考えていきます。

その準備として、本節では**損失関数**と呼ばれる数理的手法を紹介して解説していきます。

## 基礎となる数理的手法と情報工学的アプローチ

| 基礎となる数理的手法 | 情報工学的アプローチ |
|:---|:---|
| 総和記号：$a_1b_1 + a_2b_2 + \cdots + a_nb_n = \sum_{k=1}^{n} a_k b_k$ | 損失関数の設計 |

## 残差行列 E の再掲

先程 $R - \hat{R} = E$ という数式を示しましたが、この値を最小化するとはつまり、計算結果となる $E$ のすべての成分の和を0に近づけるということです。

$$E = R - \hat{R} = R - PQ$$

残差行列 $E$ の各要素は：

$$e_{u,i} = r_{u,i} - (p_{u,1}q_{1,i} + p_{u,2}q_{2,i}) = r_{u,i} - \sum_{d=1}^{2} p_{u,d}q_{d,i}$$

In [ ]:
# 残差行列の具体例
print("=== 残差行列 E の計算 ===")
print()

# 評価値行列
R = np.array([
    [2, 3, 0, 5],
    [2, 5, 0, 5],
    [0, 3, 4, 4],
    [4, 2, 3, 0]
])

print("評価値行列 R:")
print(R)
print()

# 仮のP, Q行列（d=2）
P = np.array([
    [0.5, 1.5],
    [0.8, 1.8],
    [1.2, 1.0],
    [1.5, 0.3]
])

Q = np.array([
    [1.0, 0.8, 1.5, 2.0],
    [1.2, 1.5, 1.0, 1.8]
])

print("ユーザー因子行列 P:")
print(P)
print()

print("アイテム因子行列 Q:")
print(Q)
print()

# 予測行列
R_hat = P @ Q
print("予測行列 R̂ = PQ:")
print(np.round(R_hat, 2))
print()

# 残差行列
E = R - R_hat
print("残差行列 E = R - R̂:")
print(np.round(E, 2))

## 損失関数 L の定義

この残差行列のすべての成分の和を0にするという点について、より考察しやすい形の数式を作ってみましょう。

残差行列 $E$ のすべての成分の和を計算しているこの数式を**損失関数** $L$ と定義しました。この損失関数は、他にも**目的関数**や**誤差関数**といった呼び方をされることもあります。

$$L = \{r_{1,1} - (p_{1,1}q_{1,1} + p_{1,2}q_{2,1})\} + \cdots + \{r_{4,4} - (p_{4,1}q_{1,4} + p_{4,2}q_{2,4})\}$$

### 総和記号 Σ を使った表現

この長い数式をシンプルに表すために、$\sum$ を用います。

$$L = \sum_{u=1}^{4} \sum_{i=1}^{4} \{r_{u,i} - (p_{u,1}q_{1,i} + p_{u,2}q_{2,i})\}$$

さらにシンプルに：

$$L = \sum_{u=1}^{4} \sum_{i=1}^{4} \left\{r_{u,i} - \left(\sum_{d=1}^{2} p_{u,d}q_{d,i}\right)\right\}$$

In [ ]:
# 損失関数の計算（単純な和）
print("=== 損失関数 L の計算（単純な和） ===")
print()

# 欠損値を除いた残差の和
mask = (R != 0)
L_simple = np.sum(E[mask])

print("残差行列 E（欠損値部分を除く）:")
E_masked = E.copy()
E_masked[~mask] = np.nan
print(np.round(E_masked, 2))
print()

print(f"損失関数 L（単純な和）= {L_simple:.3f}")
print()
print("問題：残差に正と負の値が混在しているため、単純な和では打ち消し合ってしまう！")

## なぜ二乗するのか

損失関数の値を小さくすることが目的なので、計算結果だけを見ると0となっていて成功しているように見えます。しかし、$r_{1,1} - (p_{1,1}q_{1,1} + p_{1,2}q_{2,1})$ の計算結果は3、$r_{1,4} - (p_{1,1}q_{1,4} + p_{1,2}q_{2,4})$ の計算結果は-5で、大きく予測が外れていることがわかります。

つまり、各要素の計算結果に正負の値が混ざると、最小化を考える上で不都合が生じるということです。

**この問題を解消するために、2乗という操作を組み込みます。**

- 負の値は2乗すると正の値、すなわちプラスになる
- 2乗した各要素をすべて足した上でその総和が最小となるようにする

$$L = \sum_{u=1}^{4} \sum_{i=1}^{4} \left\{r_{u,i} - \left(\sum_{d=1}^{2} p_{u,d}q_{d,i}\right)\right\}^2$$

In [ ]:
# 二乗誤差の損失関数
print("=== 損失関数 L の計算（二乗誤差） ===")
print()

# 欠損値を除いた二乗誤差の和
L_squared = np.sum((E[mask]) ** 2)

print("残差の二乗:")
E_squared = E ** 2
E_squared_masked = E_squared.copy()
E_squared_masked[~mask] = np.nan
print(np.round(E_squared_masked, 2))
print()

print(f"損失関数 L（二乗誤差の和）= {L_squared:.3f}")
print()
print("二乗することで:")
print("  1. 正負の打ち消し合いが起きない")
print("  2. 大きな誤差ほどペナルティが大きくなる")
print("  3. 微分が容易になる（後述）")

In [ ]:
# 損失関数の実装
def compute_loss(R, P, Q):
    """
    二乗誤差の損失関数を計算する
    
    Parameters:
    -----------
    R : numpy.ndarray
        評価値行列（0は欠損値）
    P : numpy.ndarray
        ユーザー因子行列
    Q : numpy.ndarray
        アイテム因子行列
    
    Returns:
    --------
    float
        損失関数の値
    """
    R_hat = P @ Q
    mask = (R != 0)
    error = R - R_hat
    loss = np.sum((error[mask]) ** 2)
    return loss

# テスト
loss = compute_loss(R, P, Q)
print(f"compute_loss関数を定義しました")
print(f"現在の損失: L = {loss:.3f}")

## Column: ハイパーパラメータ

導出した損失関数について、$\sum_{d=1}^{2} p_{u,d}q_{d,i}$ には潜在因子を示す $d$ という文字を用いています。この点について思い出していただきたいことがあります。それは、行列 $P$, $Q$ の「形」についてです。

- $R$ は $4 \times 4$ 行列であることに対し、$P$ は $4 \times 2$ 行列、$Q$ は $2 \times 4$ 行列としました
- $4 \times 2$ 行列と $2 \times 4$ 行列の積は $4 \times 4$ 行列となり、$R$ と同じ形になる

$$P = \begin{pmatrix} p_{1,1} & p_{1,2} \\ p_{2,1} & p_{2,2} \\ p_{3,1} & p_{3,2} \\ p_{4,1} & p_{4,2} \end{pmatrix}, \quad Q = \begin{pmatrix} q_{1,1} & q_{1,2} & q_{1,3} & q_{1,4} \\ q_{2,1} & q_{2,2} & q_{2,3} & q_{2,4} \end{pmatrix}$$

そもそも、$R$ と同じ形にしたいのであれば、$4 \times 1$ 行列と $1 \times 4$ 行列の積や、$4 \times 3$ 行列と $3 \times 4$ 行列の積でも問題はありません。

行列因子分解において、先程の $d$ に該当する部分は、**数理モデルの設計者が定める値**です。このような値のことを**ハイパーパラメータ**と呼びます。

In [ ]:
# ハイパーパラメータ d の影響
print("=== ハイパーパラメータ d の影響 ===")
print()

print("潜在因子数 d を変えると、P, Q の行列サイズが変わります:")
print()

for d in [1, 2, 3, 4]:
    print(f"d = {d}:")
    print(f"  P: 4×{d} 行列 ({4*d} 個のパラメータ)")
    print(f"  Q: {d}×4 行列 ({d*4} 個のパラメータ)")
    print(f"  合計: {4*d + d*4} 個のパラメータ")
    print()

print("d が大きいほど表現力は高まるが、過学習のリスクも高まる。")
print("d が小さいほど汎化性能は高まるが、表現力が不足する可能性がある。")

---
# 2-15 損失関数を最適化する ー最小二乗法と微分・偏微分ー

今一度、目的に立ち返りましょう。ここでの目的は損失関数 $L$ を最小化することです。

## 目的の書き換えの流れ

$$R = PQ \text{ となるような潜在因子行列 } P, Q \text{ を見つける}$$
$$\downarrow$$
$$\text{残差行列 } E \text{ を最小化する}$$
$$\downarrow$$
$$\text{残差行列 } E \text{ の各要素を2乗し、それらの総和（}\sum\text{）を計算した結果が}$$
$$\text{最小となる行列 } P, Q \text{ を見つける}$$

この「各要素を2乗し、それらの総和の値を最小化する」アプローチを**最小二乗法**と呼びます。

## 損失関数のグラフイメージ

式 (2-26) で2乗されている要素 $\{r_{u,i} - (\sum_{d=1}^{2} p_{u,d}q_{d,i})\}^2$ の中身は行列 $P$, $Q$ のそれぞれの要素であり、$p_{u,d}$, $q_{d,i}$ は1次の項です。

ということは、$\{r_{u,i} - (\sum_{d=1}^{2} p_{u,d}q_{d,i})\}^2$ の計算結果は**2次以上の多項式**だと考えられます。

つまり、この数式をグラフにすると、**曲線**あるいは**曲面**を描くはずです。

損失関数 $L$ の中身に注目して式 (2-26') を定義しましょう：

$$l = \left\{r_{u,i} - \left(\sum_{d=1}^{2} p_{u,d}q_{d,i}\right)\right\}^2$$

In [ ]:
# 図2.40: 損失関数のイメージ（2次関数）
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 左: 1変数の場合（放物線）
ax1 = axes[0]
x = np.linspace(-3, 3, 100)
y = x ** 2
ax1.plot(x, y, 'b-', linewidth=2)
ax1.axhline(y=0, color='k', linewidth=0.5)
ax1.axvline(x=0, color='k', linewidth=0.5)

# 最小点を示す
ax1.plot(0, 0, 'ro', markersize=10, label='最小点')
ax1.annotate('最小値', xy=(0, 0), xytext=(0.5, 2),
            arrowprops=dict(arrowstyle='->', color='red'),
            fontsize=12, color='red')

ax1.set_xlabel('パラメータ値', fontsize=12)
ax1.set_ylabel('損失 l', fontsize=12)
ax1.set_title('1変数の場合：放物線', fontsize=12)
ax1.legend()
ax1.grid(True, alpha=0.3)

# 右: 2変数の場合（曲面）
ax2 = axes[1]
from mpl_toolkits.mplot3d import Axes3D
ax2 = fig.add_subplot(122, projection='3d')

x = np.linspace(-2, 2, 50)
y = np.linspace(-2, 2, 50)
X, Y = np.meshgrid(x, y)
Z = X**2 + Y**2

ax2.plot_surface(X, Y, Z, cmap='viridis', alpha=0.8)
ax2.scatter([0], [0], [0], color='red', s=100, label='最小点')

ax2.set_xlabel('パラメータ1')
ax2.set_ylabel('パラメータ2')
ax2.set_zlabel('損失 L')
ax2.set_title('2変数の場合：曲面（お椀型）', fontsize=12)

plt.suptitle('図2.40: 損失関数のグラフイメージ', fontsize=14)
plt.tight_layout()
plt.show()

print("損失関数は「お椀」のような形をしており、最も低い点（最小値）を探すのが目標です。")

## 微分とは

ここで、行列 $P$, $Q$ のそれぞれの要素 $p_{u,d}$, $q_{d,i}$ は、現時点ではどのような値なのかわからない**未知数**です。

$l = \{r_{u,i} - (\sum_{d=1}^{2} p_{u,d}q_{d,i})\}^2$ という数式は2つの未知数 $p_{u,d}$, $q_{d,i}$ を含む**関数**だと言えます。

グラフの「底」の部分こそが、求めたい最小値だと言えます。しかし、グラフで描写して人間が目で見たなら「ここが最小値だ」と容易に判断できますが、このような判断をコンピュータに実行させる場合、目で見て判断するといった手法は採用できません。

**ここで登場するのが微分です。** 微分とは、簡単に言えばある関数の**傾き**を計算する手法です。

## Lesson: 微分

微分とは、数学及び物理学史上の天才として名高い**アイザック・ニュートン**（Sir Isaac Newton, 1642 - 1727）と**ゴットフリート・ライプニッツ**（Gottfried Wilhelm Leibniz, 1646 - 1716）によって発明・確立された数理的手法です。

微分は変化の「瞬間」を捉え、解析することを可能にします。

### 微分係数と導関数

- **微分係数**とは、先述した「傾き」を示す具体的な「値」のこと
- ある関数 $y = f(x)$ において、$x = a$ から $x = a + h$ に変化したとき、$y = f(x)$ の値がどのように変化するのかを考える

$$f'(a) = \lim_{h \to 0} \frac{f(a+h) - f(a)}{h}$$

### 平均変化率と微分係数

図2.42で、$x = a$ から $x = a + h$ に変化したとき $y$ の値も $f(a)$ から $f(a+h)$ に変化しています。このとき、$y = f(x)$ の**平均変化率**は $\frac{f(a+h) - f(a)}{h}$ と表すことができます。

微分係数を求めるということは、この $x$, $y$ の増加率の双方に影響している $h$ を限りなく0に近づけるということです。

In [ ]:
# 図2.42, 2.43: 微分係数のイメージ
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 左: 平均変化率
ax1 = axes[0]
x = np.linspace(0, 3, 100)
y = 0.5 * x ** 2
ax1.plot(x, y, 'b-', linewidth=2, label='$y = f(x)$')

# 点aとa+hを示す
a = 1.0
h = 1.0
fa = 0.5 * a ** 2
fah = 0.5 * (a + h) ** 2

ax1.plot([a, a], [0, fa], 'g--', alpha=0.5)
ax1.plot([a+h, a+h], [0, fah], 'g--', alpha=0.5)
ax1.plot([a, a+h], [fa, fa], 'r--', alpha=0.5, label='$x$の増加量 $h$')
ax1.plot([a+h, a+h], [fa, fah], 'orange', linestyle='--', alpha=0.5, label='$y$の増加量 $f(a+h)-f(a)$')

# 割線
ax1.plot([a, a+h], [fa, fah], 'r-', linewidth=2, label='割線')
ax1.plot(a, fa, 'ko', markersize=8)
ax1.plot(a+h, fah, 'ko', markersize=8)

ax1.annotate('$a$', (a, -0.2), fontsize=12, ha='center')
ax1.annotate('$a+h$', (a+h, -0.2), fontsize=12, ha='center')
ax1.annotate('$f(a)$', (a-0.15, fa), fontsize=10, ha='right')
ax1.annotate('$f(a+h)$', (a+h+0.1, fah), fontsize=10, ha='left')

ax1.set_xlim(-0.2, 3)
ax1.set_ylim(-0.5, 5)
ax1.set_xlabel('$x$', fontsize=12)
ax1.set_ylabel('$y$', fontsize=12)
ax1.set_title('図2.42: 平均変化率', fontsize=12)
ax1.legend(loc='upper left')
ax1.grid(True, alpha=0.3)

# 右: h→0の極限（接線）
ax2 = axes[1]
ax2.plot(x, y, 'b-', linewidth=2, label='$y = f(x)$')

# 接線
a = 1.0
fa = 0.5 * a ** 2
slope = a  # f'(x) = x なので x=1 での傾きは1

x_tangent = np.linspace(0, 2.5, 100)
y_tangent = slope * (x_tangent - a) + fa
ax2.plot(x_tangent, y_tangent, 'r-', linewidth=2, label=f'接線 (傾き={slope})')
ax2.plot(a, fa, 'ro', markersize=10)

ax2.annotate('接点', (a, fa), xytext=(a+0.3, fa+0.5),
            arrowprops=dict(arrowstyle='->', color='red'),
            fontsize=12, color='red')

ax2.set_xlim(-0.2, 3)
ax2.set_ylim(-0.5, 5)
ax2.set_xlabel('$x$', fontsize=12)
ax2.set_ylabel('$y$', fontsize=12)
ax2.set_title('図2.43: $h$→0の極限で接線になる', fontsize=12)
ax2.legend(loc='upper left')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("微分係数 f'(a) は、x = a における接線の傾きを表します。")

### 微分係数の計算例

$f(x) = \frac{1}{2}x^2 + 1$ について、$x = 1$ における微分係数 $f'(1)$ を求めてみましょう。

$$f'(1) = \lim_{h \to 0} \frac{f(1+h) - f(1)}{h}$$

$$= \lim_{h \to 0} \frac{\{\frac{1}{2}(1+h)^2 + 1\} - \{\frac{1}{2}(1)^2 + 1\}}{h}$$

$$= \lim_{h \to 0} \frac{\{\frac{1}{2} + h + \frac{1}{2}h^2 + 1\} - \frac{3}{2}}{h}$$

$$= \lim_{h \to 0} \frac{\frac{1}{2}h^2 + h}{h}$$

$$= \lim_{h \to 0} \left(\frac{1}{2}h + 1\right) = 1$$

In [ ]:
# 図2.44: 微分係数及び接線の方程式の導出例
fig, ax = plt.subplots(figsize=(8, 6))

x = np.linspace(0, 2.5, 100)
y = 0.5 * x ** 2 + 1

ax.plot(x, y, 'b-', linewidth=2, label='$y = \\frac{1}{2}x^2 + 1$')

# x=1での接線
a = 1.0
fa = 0.5 * a**2 + 1  # = 1.5
slope = 1  # f'(1) = 1

x_tangent = np.linspace(0, 2.5, 100)
y_tangent = slope * (x_tangent - a) + fa
ax.plot(x_tangent, y_tangent, 'r-', linewidth=2, label='$y = x + \\frac{1}{2}$（接線）')

ax.plot(a, fa, 'ro', markersize=10)
ax.annotate(f'接点 (1, {fa})', (a, fa), xytext=(1.3, 2.2),
            arrowprops=dict(arrowstyle='->', color='red'),
            fontsize=11, color='red')

ax.set_xlim(-0.2, 2.5)
ax.set_ylim(0, 4)
ax.set_xlabel('$x$', fontsize=12)
ax.set_ylabel('$y$', fontsize=12)
ax.set_title('図2.44: 微分係数及び接線の方程式の導出例', fontsize=12)
ax.legend(loc='upper left')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("f'(1) = 1 より、x=1 における接線の傾きは 1")
print("接線の方程式: y = x + 1/2")

### 微分係数と導関数の違い

ここまでの解説で示した通り、微分係数とは接線の傾きであり、具体的な「値」を示すものです。一方、「微分する」とは「微分係数を求める」というよりも、この傾きそのものを「関数として求める」ことを指すことが通常です。その関数のことを**導関数**と呼びます。

$$\text{微分係数}: f'(a) = \lim_{h \to 0} \frac{f(a+h) - f(a)}{h}$$

$$\text{導関数}: f'(x) = \lim_{h \to 0} \frac{f(x+h) - f(x)}{h}$$

着目すべきは、定数 $a$ と変数 $x$ のどちらが使われているかという点です。

- **微分係数**は定数 $a$ が使われているので、計算結果は当然ながら**定数**となる
- **導関数**は変数 $x$ が用いられているため、計算結果は変数 $x$ が含まれる**関数**となる

## Lesson: 微分・偏微分の計算例

実際の微分計算では以下の公式に当てはめて考えることが一般的であり、計算も簡単になります。

$$(x^n)' = nx^{n-1}$$

### 具体的な計算例

$f(x) = x$, $g(x) = 3x^2$, $h(x) = 4x^3$ をそれぞれ $x$ について微分すると：

$$\frac{d}{dx}f(x) = 1 \times x^{1-1} = 1 \times x^0 = 1$$

$$\frac{d}{dx}g(x) = 2 \times 3 \times x^{2-1} = 6 \times x^1 = 6x$$

$$\frac{d}{dx}h(x) = 3 \times 4 \times x^{3-1} = 12 \times x^2 = 12x^2$$

In [ ]:
# 微分の計算例
print("=== 微分の計算例 ===")
print()

print("公式: (x^n)' = n × x^(n-1)")
print()

# 関数と導関数のペア
functions = [
    ("f(x) = x", "f'(x) = 1"),
    ("g(x) = 3x²", "g'(x) = 6x"),
    ("h(x) = 4x³", "h'(x) = 12x²"),
]

for f, df in functions:
    print(f"{f:20} → {df}")

print()

# yで微分した場合（yを含まない関数）
print("y について微分した場合（y を含まない関数）:")
print("d/dy f(x) = 0,  d/dy g(x) = 0,  d/dy h(x) = 0")
print()
print("→ 微分する変数と関係ない変数だけの項や定数項は、微分すると0になる")

## 偏微分

$f(x, y) = 3x^2 + 2y + 1$ という2変数関数を $x$ について微分します。$x$ を含む項は1項目の $3x^2$ のみなので次のように計算されます。

$$\frac{\partial}{\partial x}f(x,y) = 2 \times 3 \times x^{2-1} = 6x$$

なお、$\frac{\partial}{\partial x}$ は多変数関数に対して1つの変数で微分する**偏微分**を示す微分記号です。

$f(x, y) = 3x^2 + 2y + 1$ を $y$ について微分すると、$y$ を含む項は2項目の $2y$ のみなので次のように計算されます：

$$\frac{\partial}{\partial y}f(x,y) = 2 \times y^{1-1} = 2$$

**つまり、微分する変数と関係ない変数だけの項や定数項は、微分すると0になるということです。**

In [ ]:
# 偏微分の計算例
print("=== 偏微分の計算例 ===")
print()

print("f(x, y) = 3x² + 2y + 1")
print()

print("x で偏微分:")
print("  ∂f/∂x = 6x")
print("  （2y + 1 は x を含まないので 0 になる）")
print()

print("y で偏微分:")
print("  ∂f/∂y = 2")
print("  （3x² + 1 は y を含まないので 0 になる）")
print()

print("=== 偏微分のポイント ===")
print("- 偏微分する変数以外は「定数」として扱う")
print("- 定数を微分すると 0 になる")
print("- 偏微分記号 ∂ は「パーシャル」と読む")

## 傾きが0のとき最小値を取る

以上を踏まえ、式 (2-26') の**接線**について考えてみましょう。グラフの接線を図2.45のように図示すると、グラフと接線の**接点**ごとに**傾き**の大きさが異なることがわかります。

この傾きが、微分によって計算される値です。図を見ればわかる通り、**傾きが0となったとき、最小値を取ります。**

In [ ]:
# 図2.45: 傾きと最小値の関係
fig, ax = plt.subplots(figsize=(10, 6))

x = np.linspace(-2, 2, 100)
y = x ** 2

ax.plot(x, y, 'b-', linewidth=2, label='$l = f(p, q)$')

# 複数の接点での接線
points = [(-1.5, (-1.5)**2), (-0.5, (-0.5)**2), (0, 0), (0.5, 0.5**2), (1.5, 1.5**2)]
labels = ['傾き:大(負)', '傾き:小(負)', '傾き:最小(0)', '傾き:小(正)', '傾き:大(正)']
colors = ['orange', 'gold', 'red', 'gold', 'orange']

for (px, py), label, color in zip(points, labels, colors):
    slope = 2 * px  # y = x^2 の導関数は 2x
    x_tan = np.linspace(px - 0.8, px + 0.8, 50)
    y_tan = slope * (x_tan - px) + py
    ax.plot(x_tan, y_tan, '-', color=color, linewidth=2, alpha=0.8)
    ax.plot(px, py, 'o', color=color, markersize=8)
    
    if px == 0:
        ax.annotate(label, (px, py), xytext=(px + 0.3, py + 0.5),
                   fontsize=10, color='red', fontweight='bold',
                   arrowprops=dict(arrowstyle='->', color='red'))
    elif px < 0:
        ax.annotate(label, (px, py), xytext=(px - 0.5, py + 1),
                   fontsize=9, ha='right')
    else:
        ax.annotate(label, (px, py), xytext=(px + 0.3, py + 1),
                   fontsize=9)

ax.axhline(y=0, color='k', linewidth=0.5)
ax.axvline(x=0, color='k', linewidth=0.5)

ax.set_xlabel('パラメータ値', fontsize=12)
ax.set_ylabel('損失 $l$', fontsize=12)
ax.set_title('図2.45: 曲線を描く「下に凸の関数」は、傾きが0に近づくほど、その点における関数の値は最小に近づく', fontsize=11)
ax.set_xlim(-2.5, 2.5)
ax.set_ylim(-0.5, 4)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---
# 2-16 計算結果を統合して数理モデルを導出する ー偏微分と総和記号Σー

では、実際に微分計算を行います。ここで微分対象となるのは損失関数 $L = \sum_{u=1}^{4} \sum_{i=1}^{4} \{r_{u,i} - (\sum_{d=1}^{2} p_{u,d}q_{d,i})\}^2$ ではなく、その $\sum$ の中身の $l = \{r_{u,i} - (\sum_{d=1}^{2} p_{u,d}q_{d,i})\}^2$ であることに留意してください。

## $l$ の展開

まず、$l$ について2乗の計算を行います。

$$l = \left\{r_{u,i} - \left(\sum_{d=1}^{2} p_{u,d}q_{d,i}\right)\right\}^2 = (r_{u,i})^2 - 2r_{u,i}\left(\sum_{d=1}^{2} p_{u,d}q_{d,i}\right) + \left(\sum_{d=1}^{2} p_{u,d}q_{d,i}\right)^2$$

これは $(a-b)^2 = a^2 - 2ab + b^2$ の形です。

## $l$ の展開（続き）

$\sum$ があると考えづらいため、和の形に戻してさらに展開します。

$$(r_{u,i})^2 - 2r_{u,i}\left(\sum_{d=1}^{2} p_{u,d}q_{d,i}\right) + \left(\sum_{d=1}^{2} p_{u,d}q_{d,i}\right)^2$$

$$= (r_{u,i})^2 - 2r_{u,i}(p_{u,1}q_{1,i} + p_{u,2}q_{2,i}) + (p_{u,1}q_{1,i} + p_{u,2}q_{2,i})^2$$

$$= (r_{u,i})^2 - 2r_{u,i} \cdot p_{u,1}q_{1,i} - 2r_{u,i} \cdot p_{u,2}q_{2,i} + (p_{u,1})^2(q_{1,i})^2 + 2p_{u,1}q_{1,i} \cdot p_{u,2}q_{2,i} + (p_{u,2})^2(q_{2,i})^2$$

数式を展開した結果、4つの未知数 $p_{u,1}$, $p_{u,2}$, $q_{1,i}$, $q_{2,i}$ が現れました。この数式を、それぞれの未知数で微分することで $l$ を最適化します。

なお、複数の未知数が存在する数式に対して1つの未知数で微分することを**偏微分**と呼びます。

## $p_{u,1}$ での偏微分

展開した式について、$p_{u,1}$ で偏微分計算を行います。これにより、未知数ごとに $l$ を最適化できます。$p_{u,1}$ に関係のない項は消えていることを確認してください。

$$\frac{\partial l}{\partial p_{u,1}} = -2r_{u,i} \cdot q_{1,i} + 2(q_{1,i})^2 p_{u,1} + 2q_{1,i} \cdot p_{u,2}q_{2,i}$$

$$= -2r_{u,i} \cdot q_{1,i} + 2q_{1,i}(p_{u,1}q_{1,i} + p_{u,2}q_{2,i})$$

$$= -2\{r_{u,i} - (p_{u,1}q_{1,i} + p_{u,2}q_{2,i})\}q_{1,i}$$

## 損失関数の偏微分の一般形

同様に他の未知数 $p_{u,2}$, $q_{1,i}$, $q_{2,i}$ でも偏微分計算を行います。比較しやすいように $\frac{\partial l}{\partial p_{u,1}}$ の計算結果も再掲します。

$$\frac{\partial L}{\partial p_{u,k}} = -2\sum_{i=1}^{4} e_{u,i}q_{k,i}$$

$$\frac{\partial L}{\partial q_{k,i}} = -2\sum_{u=1}^{4} e_{u,i}p_{u,k}$$

ただし、$e_{u,i} = r_{u,i} - \sum_{d=1}^{2} p_{u,d}q_{d,i}$（残差）

In [ ]:
# 偏微分の計算結果の確認
print("=== 損失関数の偏微分 ===")
print()

print("残差: e_{u,i} = r_{u,i} - Σ p_{u,d}q_{d,i}")
print()

print("P の要素 p_{u,k} での偏微分:")
print("  ∂L/∂p_{u,k} = -2 Σ e_{u,i} × q_{k,i}")
print("             = -2 × (残差) × (対応するQの要素)")
print()

print("Q の要素 q_{k,i} での偏微分:")
print("  ∂L/∂q_{k,i} = -2 Σ e_{u,i} × p_{u,k}")
print("             = -2 × (残差) × (対応するPの要素)")
print()

print("=== 偏微分の意味 ===")
print("これらの偏微分結果が「勾配」を表します。")
print("勾配の方向に沿ってパラメータを更新することで、損失を減らせます。")

---
# 2-17 更新式を設計して予測値を推計する ー勾配降下法ー

本章の総仕上げです。前節で導出した式 (2-31) を用いて、勾配降下法によって損失関数を最小化するために、ユーザー因子行列及びアイテム因子行列の各要素の**更新式**を定義します。

## 情報工学的アプローチ

| 損失関数の偏微分計算 | 勾配降下法によるユーザー因子行列/アイテム因子行列の各要素の更新式 |
|:---|:---|
| $\frac{\partial L}{\partial p_{u,k}} = -2\sum_{i=1}^{4} e_{u,i}q_{k,i}$ | $p_{u,k}^{(n)} = p_{u,k}^{(n-1)} + 2\eta \sum_{i=1}^{4} e_{u,i}q_{k,i}^{(n-1)}$ |
| $\frac{\partial L}{\partial q_{k,i}} = -2\sum_{u=1}^{4} e_{u,i}p_{u,k}$ | $q_{k,i}^{(n)} = q_{k,i}^{(n-1)} + 2\eta \sum_{u=1}^{4} e_{u,i}p_{u,k}^{(n-1)}$ |

ここで：
- $n$: 自然数（更新回数）
- $p_{u,k}^{(n)}$: $n$回更新後のユーザー因子行列 $P$ の $u$ 行 $k$ 列目の要素
- $q_{k,i}^{(n)}$: $n$回更新後のアイテム因子行列 $Q$ の $k$ 行 $i$ 列目の要素

## 傾きの方向に応じた更新

$\frac{\partial L}{\partial p_{u,k}}$ 及び $\frac{\partial L}{\partial q_{k,i}}$ の値が0より大きい、または小さければ最小化の余地があることを意味します。

- **傾きが正（右上がり）の場合**：$p_{u,k}$ の値を**小さくする**
- **傾きが負（右下がり）の場合**：$p_{u,k}$ の値を**大きくする**

これにより、接線の傾きを0に近づければよいことになります。

In [ ]:
# 図2.48, 2.49: 傾きの方向に応じた更新
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

x = np.linspace(-2, 2, 100)
y = x ** 2

# 左：傾きが正の場合 → 値を小さくする
ax1 = axes[0]
ax1.plot(x, y, 'b-', linewidth=2)

# 現在位置（傾きが正）
p0 = 1.5
ax1.plot(p0, p0**2, 'ro', markersize=12, label='現在位置 $p_{u,k}^{(0)}$')
ax1.axvline(x=p0, color='r', linestyle='--', alpha=0.5)

# 更新後の位置
p1 = 0.8
ax1.plot(p1, p1**2, 'go', markersize=12, label='更新後 $p_{u,k}^{(1)}$')
ax1.axvline(x=p1, color='g', linestyle='--', alpha=0.5)

# 矢印
ax1.annotate('', xy=(p1, 1), xytext=(p0, 1),
            arrowprops=dict(arrowstyle='->', color='purple', lw=2))
ax1.text((p0+p1)/2, 1.3, '値を小さくする', ha='center', fontsize=10, color='purple')

ax1.set_xlabel('$p_{u,k}$', fontsize=12)
ax1.set_ylabel('損失 $L$', fontsize=12)
ax1.set_title('図2.48: 傾きが正の方向であれば、$p_{u,k}$の値を小さくする', fontsize=11)
ax1.legend()
ax1.grid(True, alpha=0.3)

# 右：傾きが負の場合 → 値を大きくする
ax2 = axes[1]
ax2.plot(x, y, 'b-', linewidth=2)

# 現在位置（傾きが負）
p0 = -1.5
ax2.plot(p0, p0**2, 'ro', markersize=12, label='現在位置 $p_{u,k}^{(0)}$')
ax2.axvline(x=p0, color='r', linestyle='--', alpha=0.5)

# 更新後の位置
p1 = -0.8
ax2.plot(p1, p1**2, 'go', markersize=12, label='更新後 $p_{u,k}^{(1)}$')
ax2.axvline(x=p1, color='g', linestyle='--', alpha=0.5)

# 矢印
ax2.annotate('', xy=(p1, 1), xytext=(p0, 1),
            arrowprops=dict(arrowstyle='->', color='purple', lw=2))
ax2.text((p0+p1)/2, 1.3, '値を大きくする', ha='center', fontsize=10, color='purple')

ax2.set_xlabel('$p_{u,k}$', fontsize=12)
ax2.set_ylabel('損失 $L$', fontsize=12)
ax2.set_title('図2.49: 傾きが負の方向であれば、$p_{u,k}$の値を大きくする', fontsize=11)
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 更新式の導出

これらの計算を実現する数理モデルを設計します。偏微分した計算結果を活用し、$p_{u,k}$ の初期値を $p_{u,k}^{(0)}$、移動後の $p_{u,k}$ の値を $p_{u,k}^{(1)}$ とすると：

$$p_{u,k}^{(1)} = p_{u,k}^{(0)} - \eta \cdot \frac{\partial L}{\partial p_{u,k}^{(0)}}$$

$$= p_{u,k}^{(0)} - \eta \left(-2\sum_{i=1}^{4} e_{u,i}q_{k,i}^{(0)}\right)$$

$$= p_{u,k}^{(0)} + 2\eta \sum_{i=1}^{4} e_{u,i}q_{k,i}^{(0)}$$

## 学習率 η（イータ）

ここで、$\eta$（eta：イータ）は**学習率**と呼ばれるハイパーパラメータで、偏微分した値をどれだけ反映させるかを調整する役割を果たします。

図2.50に示すように、偏微分した計算結果の値が大き過ぎると、学習率 $\eta$ を用いない場合、最小値となる $p_{u,k}$ の値を飛び越えてしまう可能性があります。

In [ ]:
# 図2.50: 学習率ηの必要性
fig, ax = plt.subplots(figsize=(10, 6))

x = np.linspace(-2, 3, 100)
y = x ** 2

ax.plot(x, y, 'b-', linewidth=2)

# 現在位置
p0 = 2.0
ax.plot(p0, p0**2, 'ro', markersize=12, label='$p_{u,k}^{(0)}$')
ax.axvline(x=p0, color='r', linestyle='--', alpha=0.5)

# 最小値
ax.plot(0, 0, 'g*', markersize=15, label='最小値')
ax.axvline(x=0, color='g', linestyle='--', alpha=0.5)

# 学習率なしで飛び越えてしまう場合
p1_bad = -1.5
ax.plot(p1_bad, p1_bad**2, 'mo', markersize=12, label='学習率なしで飛び越え')
ax.annotate('', xy=(p1_bad, 0.5), xytext=(p0, 0.5),
            arrowprops=dict(arrowstyle='->', color='purple', lw=2))

# 警告テキスト
ax.text(0.25, 3, '学習率ηを用いないと\n最小値を飛び越えてしまう\n可能性がある', 
        fontsize=11, ha='center', 
        bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.8))

ax.set_xlabel('$p_{u,k}$', fontsize=12)
ax.set_ylabel('損失 $L$', fontsize=12)
ax.set_title('図2.50: 学習率ηの必要性', fontsize=12)
ax.legend(loc='upper right')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("学習率 η は通常 0.1〜0.001 程度の値で設定されることが多い。")

## 一般化した更新式

$n$ を自然数とすると、$n$ 回の更新を行った $p_{u,k}^{(n)}$ と $q_{k,i}^{(n)}$ は次の数式で計算されます。これが、ユーザー因子行列及びアイテム因子行列の各要素について、一般化した更新式です。

$$p_{u,k}^{(n)} = p_{u,k}^{(n-1)} + 2\eta \sum_{i=1}^{4} e_{u,i}q_{k,i}^{(n-1)}$$

$$q_{k,i}^{(n)} = q_{k,i}^{(n-1)} + 2\eta \sum_{u=1}^{4} e_{u,i}p_{u,k}^{(n-1)}$$

この試行を繰り返すことで、$p_{u,k}$, $q_{k,i}$ の値を最適化します。

---
# 2-18 勾配降下法の計算例

ここまで導出してきた内容をもとに、実際に勾配降下法の計算を実行してみましょう。通常はプログラミングによって計算を実行しますが、本書では具体的な計算プロセスをできる限り記述することで、数理的な理解を目指します。

ここでも、具体例として度々登場している以下の評価値行列（式 (2-16)）を用います。

$$R = \begin{pmatrix} 2 & 3 & 0 & 5 \\ 2 & 5 & 0 & 5 \\ 0 & 3 & 4 & 4 \\ 4 & 2 & 3 & 0 \end{pmatrix}$$

In [ ]:
# 勾配降下法の計算例
print("=== 勾配降下法の計算例 ===")
print()

# 評価値行列
R = np.array([
    [2, 3, 0, 5],
    [2, 5, 0, 5],
    [0, 3, 4, 4],
    [4, 2, 3, 0]
])

print("評価値行列 R:")
print(R)
print()

# 初期値（教科書の例に合わせる）
P = np.array([
    [0.37, 0.95],
    [0.73, 0.60],
    [0.16, 0.16],
    [0.06, 0.87]
])

Q = np.array([
    [0.60, 0.71, 0.02, 0.97],
    [0.83, 0.21, 0.18, 0.18]
])

print("初期ユーザー因子行列 P:")
print(P)
print()

print("初期アイテム因子行列 Q:")
print(Q)
print()

# 予測行列 R̂ = PQ
R_hat = P @ Q
print("予測行列 R̂ = PQ:")
print(np.round(R_hat, 2))

In [ ]:
# 残差行列と損失関数の計算
print("=== 残差行列 E ===")
print()

# 残差行列
E = R - R_hat
print("E = R - R̂:")
print(np.round(E, 2))
print()

# 損失関数（欠損値を除く）
mask = (R != 0)
loss = np.sum((E[mask]) ** 2)
print(f"初期損失: L = {loss:.4f}")

In [ ]:
# 行列因子分解クラスの実装
class MatrixFactorization:
    """
    行列因子分解による協調フィルタリング
    
    Parameters:
    -----------
    n_factors : int
        潜在因子の数（ハイパーパラメータ d）
    learning_rate : float
        学習率 η
    n_epochs : int
        更新回数
    """
    
    def __init__(self, n_factors=2, learning_rate=0.01, n_epochs=100):
        self.n_factors = n_factors
        self.learning_rate = learning_rate
        self.n_epochs = n_epochs
        self.P = None
        self.Q = None
        self.loss_history = []
    
    def fit(self, R):
        """
        評価値行列 R から P, Q を学習する
        """
        n_users, n_items = R.shape
        
        # 初期値をランダムに設定
        np.random.seed(42)
        self.P = np.random.rand(n_users, self.n_factors)
        self.Q = np.random.rand(self.n_factors, n_items)
        
        # マスク行列（欠損値の位置）
        mask = (R != 0)
        
        # 勾配降下法による更新
        for epoch in range(self.n_epochs):
            # 予測値と残差
            R_hat = self.P @ self.Q
            E = R - R_hat
            
            # 損失を記録
            loss = np.sum((E[mask]) ** 2)
            self.loss_history.append(loss)
            
            # P の更新
            for u in range(n_users):
                for k in range(self.n_factors):
                    gradient = 0
                    for i in range(n_items):
                        if mask[u, i]:
                            gradient += E[u, i] * self.Q[k, i]
                    self.P[u, k] += 2 * self.learning_rate * gradient
            
            # Q の更新
            for k in range(self.n_factors):
                for i in range(n_items):
                    gradient = 0
                    for u in range(n_users):
                        if mask[u, i]:
                            gradient += E[u, i] * self.P[u, k]
                    self.Q[k, i] += 2 * self.learning_rate * gradient
        
        return self
    
    def predict(self):
        """
        予測行列 R̂ = PQ を返す
        """
        return self.P @ self.Q
    
    def plot_loss(self):
        """
        損失の推移をプロット
        """
        plt.figure(figsize=(10, 5))
        plt.plot(self.loss_history, 'b-', linewidth=2)
        plt.xlabel('エポック数', fontsize=12)
        plt.ylabel('損失 L', fontsize=12)
        plt.title('勾配降下法による損失の推移', fontsize=14)
        plt.grid(True, alpha=0.3)
        plt.show()

print("MatrixFactorizationクラスを定義しました")

In [ ]:
# 行列因子分解の実行
print("=== 行列因子分解の実行 ===")
print()

# モデルの作成と学習
mf = MatrixFactorization(n_factors=2, learning_rate=0.01, n_epochs=500)
mf.fit(R)

print(f"学習完了！")
print(f"初期損失: {mf.loss_history[0]:.4f}")
print(f"最終損失: {mf.loss_history[-1]:.4f}")
print()

# 損失の推移をプロット
mf.plot_loss()

In [ ]:
# 学習結果の確認
print("=== 学習結果 ===")
print()

print("学習後のユーザー因子行列 P:")
print(np.round(mf.P, 3))
print()

print("学習後のアイテム因子行列 Q:")
print(np.round(mf.Q, 3))
print()

# 予測行列
R_hat = mf.predict()
print("予測行列 R̂ = PQ:")
print(np.round(R_hat, 2))
print()

print("元の評価値行列 R:")
print(R)
print()

print("差分（既知の評価値のみ）:")
E = R - R_hat
E_display = np.where(R != 0, np.round(E, 2), '-')
print(E_display)

In [ ]:
# 欠損値の予測結果を可視化
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 左：元の評価値行列
ax1 = axes[0]
im1 = ax1.imshow(np.where(R == 0, np.nan, R), cmap='YlOrRd', vmin=1, vmax=5)
ax1.imshow(np.where(R == 0, 1, np.nan), cmap='gray', vmin=0, vmax=1, alpha=0.3)

for i in range(4):
    for j in range(4):
        if R[i, j] == 0:
            ax1.text(j, i, '?', ha='center', va='center', fontsize=14, color='gray')
        else:
            ax1.text(j, i, str(R[i, j]), ha='center', va='center', fontsize=14)

ax1.set_xticks(range(4))
ax1.set_yticks(range(4))
ax1.set_xticklabels(['item1', 'item2', 'item3', 'item4'])
ax1.set_yticklabels(['user1', 'user2', 'user3', 'user4'])
ax1.set_title('元の評価値行列 R（?=欠損値）')

# 右：予測行列
ax2 = axes[1]
R_hat = mf.predict()
im2 = ax2.imshow(R_hat, cmap='YlOrRd', vmin=1, vmax=5)

for i in range(4):
    for j in range(4):
        if R[i, j] == 0:
            ax2.text(j, i, f'{R_hat[i,j]:.1f}', ha='center', va='center', 
                    fontsize=12, color='blue', fontweight='bold')
        else:
            ax2.text(j, i, f'{R_hat[i,j]:.1f}', ha='center', va='center', fontsize=12)

ax2.set_xticks(range(4))
ax2.set_yticks(range(4))
ax2.set_xticklabels(['item1', 'item2', 'item3', 'item4'])
ax2.set_yticklabels(['user1', 'user2', 'user3', 'user4'])
ax2.set_title('予測行列 R̂ = PQ（青字=欠損値の予測）')
plt.colorbar(im2, ax=ax2, label='評価値')

plt.tight_layout()
plt.show()

print("欠損値の予測結果:")
print(f"  user1のitem3: {R_hat[0, 2]:.2f}")
print(f"  user2のitem3: {R_hat[1, 2]:.2f}")
print(f"  user3のitem1: {R_hat[2, 0]:.2f}")
print(f"  user4のitem4: {R_hat[3, 3]:.2f}")

---
# 第2章 まとめ

## 本章で学んだ主要な概念

### 1. 評価値行列 R
- ユーザー×アイテムの行列で評価値を表現
- 0は欠損値（未評価）を示す

### 2. 協調フィルタリング
- **ユーザーベース**: 類似ユーザーの評価から予測
- **アイテムベース**: 類似アイテムの評価から予測
- コサイン類似度で「似ている」を定量化

### 3. コサイン類似度
$$\cos(a, b) = \frac{\vec{a} \cdot \vec{b}}{|\vec{a}||\vec{b}|}$$
- 中心化により評価のバイアスを除去
- 指示関数で欠損値を除外

### 4. 行列因子分解（Matrix Factorization）
$$R \approx P \times Q$$
- $P$: ユーザー因子行列（$U \times d$）
- $Q$: アイテム因子行列（$d \times I$）
- $d$: 潜在因子数（ハイパーパラメータ）

### 5. 損失関数と最小二乗法
$$L = \sum_{u,i} \left\{r_{u,i} - \sum_d p_{u,d}q_{d,i}\right\}^2$$
- 二乗することで正負の打ち消しを防ぐ

### 6. 勾配降下法
$$p_{u,k}^{(n)} = p_{u,k}^{(n-1)} + 2\eta \sum_i e_{u,i}q_{k,i}^{(n-1)}$$
- $\eta$: 学習率（ハイパーパラメータ）
- 偏微分で勾配を計算し、パラメータを更新

In [ ]:
# 本Notebookで実装した関数・クラスのまとめ
print("=== 本Notebookで実装した関数・クラス ===")
print()

print("【関数】")
print("1. cosine_similarity(a, b)")
print("   - 基本的なコサイン類似度")
print()
print("2. cosine_similarity_centered(a, b)")
print("   - 中心化付きコサイン類似度（ユーザーベース）")
print()
print("3. cosine_similarity_item_based(R, item_i, item_j)")
print("   - アイテムベースのコサイン類似度")
print()
print("4. compute_loss(R, P, Q)")
print("   - 二乗誤差の損失関数")
print()

print("【クラス】")
print("5. MatrixFactorization")
print("   - __init__(n_factors, learning_rate, n_epochs)")
print("   - fit(R): 勾配降下法で学習")
print("   - predict(): 予測行列を返す")
print("   - plot_loss(): 損失の推移をプロット")
print()

print("=== ハイパーパラメータ ===")
print("- d (n_factors): 潜在因子の数")
print("- η (learning_rate): 学習率")
print("- n_epochs: 更新回数")

---
# まとめ（Part 4）

## 2-15〜2-16で学んだこと

### 微分とは
- 関数の**傾き**を計算する手法
- 微分係数 $f'(a)$：具体的な点での傾き（値）
- 導関数 $f'(x)$：傾きを表す関数

### 微分の公式
$$(x^n)' = nx^{n-1}$$

### 偏微分
- 多変数関数に対して1つの変数で微分
- 他の変数は定数として扱う
- 記号：$\frac{\partial}{\partial x}$

### 損失関数の偏微分結果
$$\frac{\partial L}{\partial p_{u,k}} = -2\sum_{i=1}^{I} e_{u,i}q_{k,i}$$

$$\frac{\partial L}{\partial q_{k,i}} = -2\sum_{u=1}^{U} e_{u,i}p_{u,k}$$

### 傾きが0のとき最小値
- 下に凸の関数では、傾き=0 の点で最小値を取る
- この性質を利用して損失関数を最小化する

## 次のステップ（教科書の続き）
- 勾配降下法によるパラメータ更新
- 学習率 $\eta$ の役割

---
# まとめ（Part 3）

## 2-14〜2-15で学んだこと

### 損失関数 L
残差行列 $E$ のすべての要素の二乗和：

$$L = \sum_{u=1}^{U} \sum_{i=1}^{I} \left\{r_{u,i} - \left(\sum_{d=1}^{D} p_{u,d}q_{d,i}\right)\right\}^2$$

### なぜ二乗するのか
- 正負の打ち消し合いを防ぐ
- 大きな誤差にペナルティを与える
- 微分が容易になる

### 最小二乗法
「各要素を2乗し、それらの総和の値を最小化する」アプローチ

### ハイパーパラメータ
- 潜在因子数 $d$ は設計者が決める値
- $d$ が大きいほど表現力は高いが過学習のリスク
- $d$ が小さいほど汎化性能は高いが表現力不足の可能性

## 次のステップ（教科書の続き）
- 微分と偏微分
- 勾配降下法によるパラメータ更新

In [ ]:
# 本Notebookで実装した関数のまとめ
print("=== 本Notebookで実装した関数 ===")
print()
print("1. cosine_similarity(a, b)")
print("   - 基本的なコサイン類似度を計算")
print()
print("2. cosine_similarity_centered(a, b)")
print("   - ユーザーベース：中心化を行ったコサイン類似度")
print("   - 欠損値（0）を自動的に除外")
print()
print("3. cosine_similarity_item_based(R, item_i, item_j)")
print("   - アイテムベース：中心化コサイン類似度")
print("   - 列ベクトルを使用")
print()
print("=== 主な概念 ===")
print()
print("協調フィルタリング:")
print("  - ユーザーベース: 似ているユーザーの評価から予測")
print("  - アイテムベース: 似ているアイテムの評価から予測")
print()
print("行列因子分解:")
print("  - R ≈ P × Q に分解")
print("  - 潜在因子を通じてユーザーとアイテムの関係を表現")
print("  - セレンディピティ（意外な発見）を促進する可能性")

In [ ]:
# 行列因子分解の具体例
print("=== 行列因子分解の具体例 ===")
print()

# 評価値行列（5×4）
R_mf = np.array([
    [5, 3, 0, 1],
    [4, 0, 3, 2],
    [0, 5, 5, 5],
    [1, 2, 3, 4],
    [1, 0, 0, 5]
])

print("評価値行列 R (5ユーザー × 4アイテム):")
print(R_mf)
print()

# 仮にP, Qが解析的に求められたとする（d=2）
P = np.array([
    [0.3, 2.0],
    [0.7, 1.5],
    [1.9, 1.9],
    [1.6, 0.1],
    [2.0, 0.1]
])

Q = np.array([
    [0.4, 1.2, 1.7, 2.5],
    [2.4, 1.4, 1.0, 0.2]
])

print("ユーザー因子行列 P (5×2):")
print(P)
print()

print("アイテム因子行列 Q (2×4):")
print(Q)
print()

# PQを計算
PQ = P @ Q

print("予測行列 PQ (5×4):")
print(np.round(PQ, 2))
print()

print("元の評価値行列Rと予測行列PQを比較:")
print("R で既に評価値がついていた箇所と、PQ の計算結果は非常に近い値となっています。")
print("これが行列因子分解の大きな特徴です。")

In [ ]:
# アイテムベースでの予測値計算
print("=== r̂_{4,4} の計算（アイテムベース） ===")
print()

# user4の評価値
r_4_2 = R[3, 1]  # user4のitem2 = 2
r_4_3 = R[3, 2]  # user4のitem3 = 3

cos_i2_i4 = item_similarity_matrix[1, 3]  # 0.090
cos_i3_i4 = item_similarity_matrix[2, 3]  # 1.000

print("図2.32: 欠損値の推計で用いる数値")
print(f"  r₄,₂ = {r_4_2} (user4のitem2評価)")
print(f"  r₄,₃ = {r_4_3} (user4のitem3評価)")
print(f"  cos(i₂, i₄) = {cos_i2_i4:.3f}")
print(f"  cos(i₃, i₄) = {cos_i3_i4:.3f}")
print()

# 予測式: r̂_{4,4} = cos(i2,i4) × r_{4,2} + cos(i3,i4) × r_{4,3}
r_hat_4_4 = cos_i2_i4 * r_4_2 + cos_i3_i4 * r_4_3

print("計算式:")
print(f"r̂₄,₄ = cos(i₂, i₄) × r₄,₂ + cos(i₃, i₄) × r₄,₃")
print(f"     = {cos_i2_i4:.3f} × {r_4_2} + {cos_i3_i4:.3f} × {r_4_3}")
print(f"     = {cos_i2_i4 * r_4_2:.3f} + {cos_i3_i4 * r_4_3:.3f}")
print(f"     = {r_hat_4_4:.3f}")
print()
print(f"予測結果: user4のitem4への評価値は {r_hat_4_4:.3f} と推計されました。")
print("→ item4はuser4にとっては「可もなく不可もなく」という評価になりそうです。")

---
# 2-19 数理モデルの違いを俯瞰する
## 協調フィルタリングと行列因子分解

これまで学んできた3つの手法を比較してみましょう。

### ① ユーザーベース協調フィルタリング
- **視点**: ミクロ（局所的）
- **考え方**: 「このユーザーに似たユーザーは誰か？」を計算
- **類似度計算**: ユーザー間のコサイン類似度
- **予測方法**: 類似ユーザーの評価値の重み付き平均

### ② アイテムベース協調フィルタリング
- **視点**: ミクロ（局所的）
- **考え方**: 「このアイテムに似たアイテムは何か？」を計算
- **類似度計算**: アイテム間のコサイン類似度
- **予測方法**: 類似アイテムの評価値の重み付き平均

### ③ 行列因子分解
- **視点**: マクロ（大域的）
- **考え方**: 評価値行列全体を低ランク行列で近似
- **最適化**: 損失関数を最小化する P と Q を学習
- **予測方法**: $\hat{R} = P \times Q$

### 図解: ミクロ視点 vs マクロ視点

```
ミクロ視点（協調フィルタリング）:
  ┌─────────────────────────────────────┐
  │     R（評価値行列）                  │
  │  ┌───┬───┬───┬───┐               │
  │  │ 2 │ 3 │ ? │ 5 │ ← User1       │
  │  ├───┼───┼───┼───┤               │
  │  │ 2 │ 5 │ ? │ 5 │ ← User2 似てる!│
  │  ├───┼───┼───┼───┤               │
  │  │ ? │ 3 │ 4 │ 4 │               │
  │  └───┴───┴───┴───┘               │
  │     ↑                             │
  │   個別の類似性を計算                │
  └─────────────────────────────────────┘

マクロ視点（行列因子分解）:
  ┌─────────────────────────────────────┐
  │  R ≈ P × Q                          │
  │                                     │
  │  全体を見て「潜在因子」を発見        │
  │  例: ジャンル傾向、好みの傾向        │
  │                                     │
  │  P: 各ユーザーの潜在因子への重み    │
  │  Q: 各アイテムの潜在因子への重み    │
  └─────────────────────────────────────┘
```


In [ ]:
# 3つの手法を実際に比較
print("=== 3つの手法の比較 ===")
print()

# 評価値行列
R = np.array([
    [2, 3, 0, 5],
    [2, 5, 0, 5],
    [0, 3, 4, 4],
    [4, 2, 3, 0]
])

print("元の評価値行列 R:")
print(R)
print()

# 予測対象: r_{4,4}（ユーザー4のアイテム4への評価）
target_user = 3  # 0-indexed
target_item = 3  # 0-indexed

print(f"予測対象: r_{{4,4}}（ユーザー4のアイテム4への評価）")
print("="*50)
print()

# ① ユーザーベース
print("① ユーザーベース協調フィルタリング")
print("-"*40)

# ユーザー4と他のユーザーの類似度
user4 = R[3]
similarities_user = []
for u in range(3):
    sim = cosine_similarity_centered(R[u], user4)
    similarities_user.append(sim)
    print(f"  sim(user4, user{u+1}) = {sim:.4f}")

# 予測値計算
mask = R[:, target_item] != 0
numerator = sum(similarities_user[u] * R[u, target_item] for u in range(3) if mask[u])
denominator = sum(abs(similarities_user[u]) for u in range(3) if mask[u])
pred_user = numerator / denominator if denominator != 0 else 0
print(f"  → 予測値: {pred_user:.4f}")
print()

# ② アイテムベース
print("② アイテムベース協調フィルタリング")
print("-"*40)

# アイテム4と他のアイテムの類似度
similarities_item = []
for i in range(3):
    sim = cosine_similarity_item_based(R, i, target_item)
    similarities_item.append(sim)
    print(f"  sim(item4, item{i+1}) = {sim:.4f}")

# 予測値計算
mask = R[target_user, :] != 0
numerator = sum(similarities_item[i] * R[target_user, i] for i in range(3) if mask[i])
denominator = sum(abs(similarities_item[i]) for i in range(3) if mask[i])
pred_item = numerator / denominator if denominator != 0 else 0
print(f"  → 予測値: {pred_item:.4f}")
print()

# ③ 行列因子分解
print("③ 行列因子分解")
print("-"*40)
mf = MatrixFactorization(n_factors=2, learning_rate=0.01, n_epochs=1000)
mf.fit(R)
R_hat = mf.predict()
pred_mf = R_hat[target_user, target_item]
print(f"  最終損失: {mf.loss_history[-1]:.4f}")
print(f"  → 予測値: {pred_mf:.4f}")
print()

# 比較まとめ
print("="*50)
print("予測値の比較")
print("="*50)
print(f"  ① ユーザーベース: {pred_user:.4f}")
print(f"  ② アイテムベース: {pred_item:.4f}")
print(f"  ③ 行列因子分解:   {pred_mf:.4f}")

---
# 2-20 本章で得られた学び

## AIの予測のプロセス

本章で学んだ「評価値予測」は、以下の4ステップで構成されます:

```
┌────────────────────────────────────────────────────────────────┐
│                    AIの予測プロセス                            │
├────────────────────────────────────────────────────────────────┤
│                                                                │
│  ① 問題を発見する                                             │
│     ├── 何を予測したいか？                                    │
│     └── 「ユーザーがアイテムにつける評価値を予測したい」       │
│                    ↓                                          │
│  ② 数理モデルを作る                                           │
│     ├── 予測のための数式を定義                                │
│     ├── 協調フィルタリング: 類似度ベースの予測式              │
│     └── 行列因子分解: R ≈ P × Q                               │
│                    ↓                                          │
│  ③ データからパラメータを推定する                             │
│     ├── 損失関数を定義                                        │
│     ├── 勾配降下法でパラメータを最適化                        │
│     └── P, Q の値を学習                                       │
│                    ↓                                          │
│  ④ 予測値を出力する                                           │
│     └── 学習したモデルで欠損値を予測                          │
│                                                                │
└────────────────────────────────────────────────────────────────┘
```

## 本章で学んだ数学

| 概念 | 記号・式 | 役割 |
|------|----------|------|
| 内積 | $\mathbf{a} \cdot \mathbf{b} = \sum_i a_i b_i$ | ベクトル間の関係を数値化 |
| ノルム | $\|\mathbf{a}\| = \sqrt{\sum_i a_i^2}$ | ベクトルの大きさ |
| コサイン類似度 | $\cos(\mathbf{a}, \mathbf{b}) = \frac{\mathbf{a} \cdot \mathbf{b}}{\|\mathbf{a}\| \|\mathbf{b}\|}$ | 類似性の尺度 |
| 行列積 | $(AB)_{ij} = \sum_k A_{ik} B_{kj}$ | 行列因子分解の基礎 |
| 損失関数 | $L = \sum_{(u,i)} (r_{u,i} - \hat{r}_{u,i})^2$ | 予測の良さの指標 |
| 偏微分 | $\frac{\partial L}{\partial p_{u,k}}$ | パラメータごとの勾配 |
| 勾配降下法 | $p_{u,k} \leftarrow p_{u,k} - \eta \frac{\partial L}{\partial p_{u,k}}$ | パラメータの更新 |

## 本章で実装した関数

```python
# 類似度計算
cosine_similarity(a, b)                    # 基本的なコサイン類似度
cosine_similarity_centered(a, b)           # 中心化コサイン類似度
cosine_similarity_item_based(R, i, j)      # アイテムベースの類似度

# 損失関数
compute_loss(R, P, Q)                      # 二乗誤差損失

# 行列因子分解
class MatrixFactorization:
    fit(R)          # 学習
    predict()       # 予測
    plot_loss()     # 損失推移の可視化
```

## 次のステップ

この数学的基礎を理解した上で、以下の発展的なトピックに進むことができます:

1. **正則化**: 過学習を防ぐための L2 正則化項の追加
2. **暗黙的フィードバック**: クリックや閲覧履歴からの推薦
3. **ディープラーニング**: ニューラルネットワークを用いた推薦システム
4. **実装への応用**: MCPサーバーへの組み込み


In [ ]:
# 本章の学習成果を確認
print("="*60)
print("       第2章 完了おめでとうございます！")
print("="*60)
print()
print("本Notebookで学んだこと:")
print()
print("  ✓ 評価値行列 R の構造と意味")
print("  ✓ コサイン類似度による類似性計算")
print("  ✓ 中心化による評価バイアスの除去")
print("  ✓ ユーザーベース協調フィルタリング")
print("  ✓ アイテムベース協調フィルタリング")
print("  ✓ 行列因子分解 R ≈ P × Q")
print("  ✓ 損失関数と最小二乗法")
print("  ✓ 微分・偏微分の基礎")
print("  ✓ 勾配降下法によるパラメータ最適化")
print()
print("これらの知識を使って、実際の推薦システムを")
print("構築する準備ができました！")
print()
print("="*60)

---
# 補足: なぜ行列因子分解が強力なのか？

## 協調フィルタリングの限界

協調フィルタリングには以下の問題があります:

### 1. スパース性問題（Sparsity Problem）
現実の評価値行列は非常にスパース（疎）です。Amazonでは数百万アイテムがありますが、
ユーザーが評価するのはその0.01%未満です。

```
理想:                        現実:
[5, 3, 4, 2, 5]              [5, 0, 0, 0, 0]
[4, 5, 3, 5, 2]              [0, 0, 0, 5, 0]
[3, 2, 5, 4, 3]              [0, 3, 0, 0, 0]
                             → 類似度計算が困難!
```

### 2. コールドスタート問題（Cold Start Problem）
- 新規ユーザー: 評価履歴がないため類似ユーザーを見つけられない
- 新規アイテム: 誰にも評価されていないため推薦できない

## 行列因子分解の強み

行列因子分解はこれらの問題を**潜在因子（Latent Factors）**で緩和します:

```
P (ユーザー因子):            Q (アイテム因子):
User1: [アクション好き, SF好き]   Item1: [アクション強, SF弱]
User2: [アクション弱, SF強]      Item2: [アクション弱, SF強]

→ User2とItem2は直接の評価がなくても、
  「SF好き」という共通因子で高い予測値が出る！
```


In [ ]:
# スパース行列での比較実験
print("=== スパース行列での手法比較 ===")
print()

# 非常にスパースな行列（実際の推薦システムに近い）
R_sparse = np.array([
    [5, 0, 0, 0, 0, 0, 4, 0],
    [0, 0, 3, 0, 0, 5, 0, 0],
    [0, 4, 0, 0, 2, 0, 0, 0],
    [0, 0, 0, 5, 0, 0, 0, 3],
    [2, 0, 0, 0, 0, 0, 0, 4],
    [0, 0, 4, 0, 0, 3, 0, 0]
])

print("スパースな評価値行列 R:")
print(R_sparse)
print()

# スパース率を計算
total = R_sparse.size
non_zero = np.count_nonzero(R_sparse)
sparsity = 1 - (non_zero / total)
print(f"スパース率: {sparsity:.1%} ({total - non_zero}/{total} が欠損)")
print()

# 行列因子分解で全ての欠損値を予測
print("行列因子分解による予測:")
mf_sparse = MatrixFactorization(n_factors=3, learning_rate=0.01, n_epochs=2000)
mf_sparse.fit(R_sparse)
R_hat_sparse = mf_sparse.predict()

print(f"最終損失: {mf_sparse.loss_history[-1]:.4f}")
print()
print("予測された完全な行列 R̂:")
print(np.round(R_hat_sparse, 1))
print()

# 元の評価値との比較
print("元の評価値（0以外）との比較:")
mask = R_sparse != 0
mae = np.mean(np.abs(R_sparse[mask] - R_hat_sparse[mask]))
print(f"平均絶対誤差 (MAE): {mae:.4f}")

---
# 補足: 正則化（Regularization）

## 過学習の問題

行列因子分解で学習を続けると、訓練データに対する損失は0に近づきます。
しかし、これは**過学習（Overfitting）**を引き起こす可能性があります。

```
過学習:
- 訓練データ: 完璧に予測 ✓
- 未知のデータ: 予測がズレる ✗
```

## L2正則化

正則化項を損失関数に追加して、パラメータが大きくなりすぎるのを防ぎます:

$$L_{\text{reg}} = \sum_{(u,i) \in \mathcal{O}} (r_{u,i} - \hat{r}_{u,i})^2 + \lambda \left( \sum_{u,k} p_{u,k}^2 + \sum_{k,i} q_{k,i}^2 \right)$$

- $\lambda$: 正則化パラメータ（ハイパーパラメータ）
- 大きな$\lambda$: 正則化が強い → 過学習を防ぐが、学習が不十分になる可能性
- 小さな$\lambda$: 正則化が弱い → 過学習のリスク


In [ ]:
# 正則化付き行列因子分解
class MatrixFactorizationRegularized:
    """
    L2正則化付き行列因子分解
    
    損失関数:
    L = Σ (r - r̂)² + λ(Σp² + Σq²)
    """
    def __init__(self, n_factors=2, learning_rate=0.01, n_epochs=100, reg_lambda=0.01):
        self.n_factors = n_factors
        self.learning_rate = learning_rate
        self.n_epochs = n_epochs
        self.reg_lambda = reg_lambda  # 正則化パラメータ
        self.P = None
        self.Q = None
        self.loss_history = []
    
    def fit(self, R):
        n_users, n_items = R.shape
        
        # 初期化
        np.random.seed(42)
        self.P = np.random.rand(n_users, self.n_factors) * 0.1
        self.Q = np.random.rand(self.n_factors, n_items) * 0.1
        
        mask = (R != 0)
        
        for epoch in range(self.n_epochs):
            R_hat = self.P @ self.Q
            E = (R - R_hat) * mask
            
            # 正則化項を含む損失
            loss = np.sum(E**2) + self.reg_lambda * (np.sum(self.P**2) + np.sum(self.Q**2))
            self.loss_history.append(loss)
            
            # 勾配計算（正則化項の微分を含む）
            grad_P = -2 * E @ self.Q.T + 2 * self.reg_lambda * self.P
            grad_Q = -2 * self.P.T @ E + 2 * self.reg_lambda * self.Q
            
            # パラメータ更新
            self.P -= self.learning_rate * grad_P
            self.Q -= self.learning_rate * grad_Q
        
        return self
    
    def predict(self):
        return self.P @ self.Q

print("MatrixFactorizationRegularized クラスを定義しました")

In [ ]:
# 正則化の効果を比較
print("=== 正則化の効果 ===")
print()

R = np.array([
    [2, 3, 0, 5],
    [2, 5, 0, 5],
    [0, 3, 4, 4],
    [4, 2, 3, 0]
])

# 正則化なし vs あり
mf_no_reg = MatrixFactorization(n_factors=2, learning_rate=0.01, n_epochs=1000)
mf_with_reg = MatrixFactorizationRegularized(n_factors=2, learning_rate=0.01, n_epochs=1000, reg_lambda=0.1)

mf_no_reg.fit(R)
mf_with_reg.fit(R)

print("正則化なし:")
print(f"  最終損失: {mf_no_reg.loss_history[-1]:.4f}")
print(f"  Pのノルム: {np.linalg.norm(mf_no_reg.P):.4f}")
print(f"  Qのノルム: {np.linalg.norm(mf_no_reg.Q):.4f}")
print()

print("正則化あり (λ=0.1):")
print(f"  最終損失: {mf_with_reg.loss_history[-1]:.4f}")
print(f"  Pのノルム: {np.linalg.norm(mf_with_reg.P):.4f}")
print(f"  Qのノルム: {np.linalg.norm(mf_with_reg.Q):.4f}")
print()

# 予測値の比較
print("予測行列の比較:")
print("\n正則化なし:")
print(np.round(mf_no_reg.predict(), 2))
print("\n正則化あり:")
print(np.round(mf_with_reg.predict(), 2))
print("\n→ 正則化ありの方がパラメータのノルムが小さく、")
print("  汎化性能が高い（未知のデータに対して良い予測をする）可能性があります。")

---
# 補足: ハイパーパラメータチューニング

行列因子分解には複数のハイパーパラメータがあります:

| パラメータ | 意味 | 典型的な範囲 |
|-----------|------|-------------|
| d (n_factors) | 潜在因子の数 | 10〜200 |
| η (learning_rate) | 学習率 | 0.001〜0.1 |
| epochs | 学習回数 | 100〜10000 |
| λ (reg_lambda) | 正則化パラメータ | 0.001〜0.1 |

これらは**交差検証（Cross Validation）**で最適値を探します。


In [ ]:
# 学習率の比較
print("=== 学習率の影響 ===")
print()

R = np.array([
    [2, 3, 0, 5],
    [2, 5, 0, 5],
    [0, 3, 4, 4],
    [4, 2, 3, 0]
])

learning_rates = [0.001, 0.01, 0.05, 0.1]
colors = ['blue', 'green', 'orange', 'red']

plt.figure(figsize=(12, 5))

for lr, color in zip(learning_rates, colors):
    mf = MatrixFactorization(n_factors=2, learning_rate=lr, n_epochs=500)
    mf.fit(R)
    plt.plot(mf.loss_history, color=color, label=f'η={lr}', alpha=0.7)

plt.xlabel('エポック数')
plt.ylabel('損失 L')
plt.title('学習率による収束速度の違い')
plt.legend()
plt.grid(True, alpha=0.3)
plt.yscale('log')  # 対数スケール
plt.tight_layout()
plt.show()

print("考察:")
print("  - 学習率が小さい (η=0.001): 収束が遅い")
print("  - 学習率が適切 (η=0.01〜0.05): スムーズに収束")
print("  - 学習率が大きすぎる (η=0.1): 不安定になる可能性")

---
# 補足: 確率的勾配降下法（SGD）

本章で実装した勾配降下法は**バッチ勾配降下法**と呼ばれます。
これは全てのデータを使って勾配を計算します。

実際の大規模システムでは、**確率的勾配降下法（SGD）**がよく使われます:

```python
# バッチ勾配降下法（本章の実装）
for epoch in range(n_epochs):
    # 全データで勾配計算
    grad = compute_gradient(all_data)
    update_parameters(grad)

# 確率的勾配降下法（SGD）
for epoch in range(n_epochs):
    for (user, item, rating) in shuffle(data):
        # 1サンプルで勾配計算
        grad = compute_gradient_single(user, item, rating)
        update_parameters(grad)
```

### SGDのメリット
1. **メモリ効率**: 全データをメモリに載せる必要がない
2. **計算速度**: 1エポックあたりの計算は早い
3. **オンライン学習**: 新しいデータが来たらすぐに学習できる

### ミニバッチSGD
バッチとSGDの中間。32〜256サンプルずつ処理:
- バッチのノイズ軽減効果
- SGDの計算効率


---
# 実践的なTips

## 1. 評価指標

推薦システムの性能を測る指標:

| 指標 | 説明 | 式 |
|------|------|----|
| RMSE | 二乗平均平方根誤差 | $\sqrt{\frac{1}{n}\sum(r - \hat{r})^2}$ |
| MAE | 平均絶対誤差 | $\frac{1}{n}\sum|r - \hat{r}|$ |
| Precision@k | 上位k件の適合率 | 関連アイテム数 / k |
| Recall@k | 上位k件の再現率 | 上位k件の関連アイテム数 / 全関連アイテム数 |

## 2. データの前処理

```python
# 評価値のスケーリング（1-5 → 0-1）
R_scaled = (R - 1) / 4

# グローバルバイアスの除去
global_mean = np.mean(R[R != 0])
R_centered = R - global_mean  # 0は変えない
```

## 3. 実装時の注意点

- **数値安定性**: 勾配爆発を防ぐため、勾配クリッピングを使う
- **早期停止**: 検証データの損失が上がり始めたら学習を止める
- **バイアス項**: ユーザーバイアス・アイテムバイアスを追加すると精度向上

```python
# バイアス付き予測
r̂_ui = μ + b_u + b_i + p_u · q_i
```


In [ ]:
# 評価指標の実装
def rmse(R_true, R_pred, mask):
    """Root Mean Squared Error"""
    error = R_true - R_pred
    return np.sqrt(np.mean(error[mask]**2))

def mae(R_true, R_pred, mask):
    """Mean Absolute Error"""
    error = R_true - R_pred
    return np.mean(np.abs(error[mask]))

# 使用例
R = np.array([
    [2, 3, 0, 5],
    [2, 5, 0, 5],
    [0, 3, 4, 4],
    [4, 2, 3, 0]
])

mf = MatrixFactorization(n_factors=2, learning_rate=0.01, n_epochs=1000)
mf.fit(R)
R_hat = mf.predict()

mask = R != 0

print("=== 評価指標 ===")
print(f"RMSE: {rmse(R, R_hat, mask):.4f}")
print(f"MAE:  {mae(R, R_hat, mask):.4f}")
print()
print("目安:")
print("  RMSE < 1.0: 良好な予測")
print("  RMSE < 0.5: 優れた予測")